# Exploring The Dimensions Search Language (DSL) - Deep Dive

This notebook takes you through all the most important features of the [Dimensions Search Language](https://docs.dimensions.ai/dsl/). 

The notebook is based on the [Query Syntax](https://docs.dimensions.ai/dsl/language.html) section of the official documentation. It can seen as an interactive version of the documentaion, as it allows to go run the queries included in that section using Python.  

## What is the Dimensions Search Language?

The DSL aims to capture the type of interaction with Dimensions data
that users are accustomed to performing graphically via the [web
application](https://app.dimensions.ai/), and enable web app developers, power users, and others to
carry out such interactions by writing query statements in a syntax
loosely inspired by SQL but particularly suited to our specific domain
and data organization.

**Note:** this notebook uses the Python programming language, however all the **DSL queries are not Python-specific** and can in fact be reused with any other API client. 



## Prerequisites

This notebook assumes you have installed the [Dimcli](https://pypi.org/project/dimcli/) library and are familiar with the *Getting Started* tutorial.


In [298]:

# @markdown # Get the API library and login
# @markdown Click the 'play' button on the left (or shift+enter) after entering your API credentials

username = "" #@param {type: "string"}
password = "" #@param {type: "string"}
endpoint = "https://app.dimensions.ai" #@param {type: "string"}


!pip install dimcli plotly tqdm -U --quiet
import dimcli
from dimcli.shortcuts import *
dimcli.login(username, password, endpoint)
dsl = dimcli.Dsl()

#
# load common libraries
import time
import sys
import json
import pandas as pd
from pandas.io.json import json_normalize
from tqdm.notebook import tqdm as progress

#
# charts libs
# import plotly_express as px
import plotly.express as px
if not 'google.colab' in sys.modules:
  # make js dependecies local / needed by html exports
  from plotly.offline import init_notebook_mode
  init_notebook_mode(connected=True)

DimCli v0.6.7 - Succesfully connected to <https://app.dimensions.ai> (method: dsl.ini file)



## Sections Index 

1. Basic query structure
2. Full-text searching
3. Field searching
4. Searching for researchers
5. Returning results 
6. Aggregations

## 1. Basic query structure

DSL queries consist of two required components: a `search` phrase that
indicates the scientific records to be searched, and one or
more `return` phrases which specify the contents and structure of the
desired results.

The simplest valid DSL query is of the form `search <source>|return <result>`:

In [299]:
%%dsldf 
search grants return  grants limit 5

Returned Grants: 5 (total = 5263527)


id  start_date     project_num  start_year title_language  \
0  grant.8690978  2021-11-30  2018-HRSI-1548        2021             en   
1  grant.8950252  2021-10-01        1301720F        2021             en   
2  grant.8715161  2021-10-01          M 2734        2021             en   
3  grant.8963889  2021-09-01          893021        2021             en   
4  grant.8964235  2021-09-01          892933        2021             en   

                                               title  \
0  APPROACH to Enriching the Real World Evidence ...   
1  Molecular mechanism of DNA double strand break...   
2                     Life as concept and as science   
3  Jet quenching for heavy-ion collisions at the LHC   
4  Scintillation Light For New Physics with Liqui...   

                                      original_title  \
0  APPROACH to Enriching the Real World Evidence ...   
1  Mécanismes moléculaires de la formation et la ...   
2                     Life as concept and as science   
3  Jet quenching for heavy-ion collisions at the LHC   
4  Scintillation Light For New Physics with Liqui...   

                                             funders language  \
0  [{'id': 'grid.484521.e', 'state_name': 'New Br...       en   
1  [{'id': 'grid.424470.1', 'linkout': ['http://w...       en   
2  [{'id': 'grid.25111.36', 'linkout': ['https://...       en   
3  [{'id': 'grid.270680.b', 'linkout': ['http://e...       en   
4  [{'id': 'grid.270680.b', 'linkout': ['http://e...       en   

          active_year                          funding_org_name    end_date  
0              [2021]  New Brunswick Health Research Foundation         NaN  
1              [2021]              Fund for Scientific Research         NaN  
2  [2021, 2022, 2023]                 FWF Austrian Science Fund  2023-09-30  
3  [2021, 2022, 2023]                       European Commission  2023-08-31  
4  [2021, 2022, 2023]                       European Commission  2023-08-31

### `search source`

A query must begin with the word `search` followed by a `source` name, i.e. the name of a type of scientific `record`, such as `grants` or `publications`.

**What are the sources available?** See the [data sources](https://docs.dimensions.ai/dsl/data-sources.html) section of the documentation. 

Alternatively, we can use the 'schema' API ([describe](https://docs.dimensions.ai/dsl/data-sources.html#metadata-api)) to return this information programmatically:

In [300]:
dsl.query("describe schema")

<dimcli.Dataset object #4874351440. Dict keys: 'sources', 'entities'>

A more useful query might also make use of the optional `for` and
`where` phrases to limit the set of records returned.

In [301]:
%%dsldf 
search grants  for "lung cancer" 
    where active_year=2000 
return  grants  limit 5

Returned Grants: 5 (total = 1728)


id  start_date  project_num    end_date  start_year  \
0  grant.2386513  2000-12-31  F32HL010455  2002-01-01        2000   
1  grant.2537116  2000-12-18  R01HL063695  2004-11-30        2000   
2  grant.2537801  2000-12-18  R01HL066221  2007-11-30        2000   
3  grant.2536777  2000-12-15  R01HL062244  2017-12-31        2000   
4  grant.2475327  2000-12-01  R01CA089614  2016-06-30        2000   

  title_language                                              title  \
0             en   ROLE OF CD44 ISOFORMS IN ENDOTHELIAL CELL DAMAGE   
1             en  ESTROGEN, ANGIOGENESIS AND ENDOTHELIAL PROGENI...   
2             en  GENETIC ANALYSIS OF EPHRIN-EPH SIGNALING IN AN...   
3             en  Synthetic Heparan Sulfate: Probing Biosynthesi...   
4             en  Redox Regulation of the rho-GTPases in the vas...   

                                      original_title  \
0   ROLE OF CD44 ISOFORMS IN ENDOTHELIAL CELL DAMAGE   
1  ESTROGEN, ANGIOGENESIS AND ENDOTHELIAL PROGENI...   
2  GENETIC ANALYSIS OF EPHRIN-EPH SIGNALING IN AN...   
3  Synthetic Heparan Sulfate: Probing Biosynthesi...   
4  Redox Regulation of the rho-GTPases in the vas...   

                                             funders language  \
0  [{'id': 'grid.279885.9', 'state_name': 'Maryla...       en   
1  [{'id': 'grid.279885.9', 'state_name': 'Maryla...       en   
2  [{'id': 'grid.279885.9', 'state_name': 'Maryla...       en   
3  [{'id': 'grid.279885.9', 'state_name': 'Maryla...       en   
4  [{'id': 'grid.48336.3a', 'state_name': 'Maryla...       en   

                                         active_year  \
0                                 [2000, 2001, 2002]   
1                     [2000, 2001, 2002, 2003, 2004]   
2   [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007]   
3  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
4  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   

                          funding_org_name  
0  National Heart Lung and Blood Institute  
1  National Heart Lung and Blood Institute  
2  National Heart Lung and Blood Institute  
3  National Heart Lung and Blood Institute  
4                National Cancer Institute

### `return` result (source or facet)

The most basic `return` phrase consists of the keyword `return` followed
by the name of a `record` or `facet` to be returned. 

This must be the
name of the `source` used in the `search` phrase, or the name of a
`facet` of that source.

In [302]:
%%dsldf
search grants for "laryngectomy" 
return grants limit 5

Returned Grants: 5 (total = 104)


funders  \
0  [{'id': 'grid.214431.1', 'longitude': -77.1868...   
1  [{'id': 'grid.54432.34', 'longitude': 139.7403...   
2  [{'id': 'grid.450640.3', 'longitude': -47.9478...   
3  [{'id': 'grid.457810.f', 'longitude': -77.111,...   
4  [{'id': 'grid.214431.1', 'longitude': -77.1868...   

                            active_year  \
0  [2019, 2020, 2021, 2022, 2023, 2024]   
1                    [2019, 2020, 2021]   
2              [2019, 2020, 2021, 2022]   
3                    [2018, 2019, 2020]   
4                          [2018, 2019]   

                                      original_title  start_year    end_date  \
0  Wearable silent speech technology to enhance i...        2019  2024-07-31   
1           喉頭がん、下咽頭がんにより喉頭摘出術を受けた患者に対する嗅覚向上プログラムの開発        2019  2021-03-31   
2              Desenvolvimento de uma prótese de voz        2019  2022-02-28   
3  EAGER: Design of an Active Voice Box Prosthesi...        2018  2020-07-31   
4                              EMG Voice Restoration        2018  2019-10-31   

   project_num             id  \
0  R01DC016621  grant.8554260   
1     19K19574  grant.8422934   
2          N/A  grant.8475371   
3      1836333  grant.7672598   
4  R43DC017097  grant.7552012   

                                    funding_org_name  start_date  \
0  National Institute on Deafness and Other Commu...  2019-08-15   
1         Japan Society for the Promotion of Science  2019-04-01   
2  National Council for Scientific and Technologi...  2019-02-18   
3                        Directorate for Engineering  2018-08-01   
4  National Institute on Deafness and Other Commu...  2018-05-16   

  title_language                                              title language  
0             en  Wearable silent speech technology to enhance i...       en  
1             ja  Development of an olfactory improvement progra...       ja  
2             pt                  Development of a voice prosthesis       pt  
3             en  EAGER: Design of an Active Voice Box Prosthesi...       en  
4             en                              EMG Voice Restoration       en

Eg let's see what are the *facets* available for the *grants* source:

In [303]:
fields = dsl.query("describe schema")['sources']['grants']['fields']
[x for x in fields if fields[x]['is_facet']]

['research_org_state_codes',
 'category_hrcs_hc',
 'category_hrcs_rac',
 'category_hra',
 'funding_org_name',
 'active_year',
 'research_org_countries',
 'funding_org_acronym',
 'category_icrp_cso',
 'category_icrp_ct',
 'language',
 'funder_countries',
 'funding_org_city',
 'researchers',
 'research_orgs',
 'category_rcdc',
 'funding_currency',
 'funders',
 'start_year',
 'research_org_cities',
 'category_bra',
 'language_title',
 'category_for']

## 2. Full-text Searching

Full-text search or keyword search finds all instances of a term
(keyword) in a document, or group of documents. 

Full text search works
by using search indexes, which can be targeting specific sections of a
document e.g. its $abstract$, $authors$, $full text$ etc...

In [304]:
%%dsldf 
search publications 
    in full_data for "Apollo 11" 
return publications limit 5

Returned Publications: 5 (total = 216069)


issue  year              id  \
0     2  2020  pub.1124928412   
1   NaN  2020  pub.1125380177   
2   NaN  2020  pub.1126068534   
3   NaN  2020  pub.1124034710   
4   NaN  2020  pub.1124265303   

                                 author_affiliations  \
0  [[{'first_name': 'Camila Aline', 'last_name': ...   
1                                                NaN   
2                                                NaN   
3  [[{'first_name': 'Bodo', 'last_name': 'Birk', ...   
4  [[{'first_name': 'Gordon', 'last_name': 'Teske...   

                                               title     type    pages volume  \
0  Fantastic Creatures and where to find them in ...  article  235-252     32   
1                    I. THE MEETING OF EAST AND WEST  chapter     1-47    NaN   
2   Karl Marx . Der 18. Brumaire des Louis Bonaparte  chapter  679-761    NaN   
3  Ausnahmezustand – Vom barocken Festspiel zur E...  chapter  291-320    NaN   
4  Chapter 8. The Period Concept and Seventeenth-...  chapter  145-162    NaN   

     journal.id                                      journal.title  
0  jour.1148021  Classica - Revista Brasileira de Estudos Cláss...  
1           NaN                                                NaN  
2           NaN                                                NaN  
3           NaN                                                NaN  
4           NaN                                                NaN

### 2.1 `in [search index]`

This optional phrase consists of the particle `in` followed by a term indicating a `search index`, specifying for example whether the search
is limited to full text, title and abstract only, or title only. 

In [305]:
%%dsldf 
search grants 
    in title_abstract_only for "something" 
return grants limit 5

Returned Grants: 5 (total = 9316)


funders  \
0  [{'id': 'grid.14467.30', 'longitude': -1.78553...   
1  [{'id': 'grid.457875.c', 'longitude': -77.1109...   
2  [{'id': 'grid.452896.4', 'longitude': 4.359973...   
3  [{'id': 'grid.457875.c', 'longitude': -77.1109...   
4  [{'id': 'grid.457810.f', 'longitude': -77.111,...   

                            active_year  \
0              [2020, 2021, 2022, 2023]   
1              [2020, 2021, 2022, 2023]   
2  [2020, 2021, 2022, 2023, 2024, 2025]   
3  [2020, 2021, 2022, 2023, 2024, 2025]   
4  [2020, 2021, 2022, 2023, 2024, 2025]   

                                      original_title  start_year    end_date  \
0       The Cosmology of the Early and Late Universe        2020  2023-09-29   
1  Decoding the Infrared Spectra of High Frequenc...        2020  2023-08-31   
2  Overcoming stellar activity in radial velocity...        2020  2025-06-30   
3  CAREER: Theory of Membrane Shape Sensing at th...        2020  2025-06-30   
4  CAREER: Modeling Human Gait to Optimize Exoske...        2020  2025-05-31   

    project_num             id  \
0  ST/T000732/1  grant.8673892   
1       1900095  grant.8966252   
2        865624  grant.8964099   
3       1945141  grant.8832391   
4       1943561  grant.8833367   

                                   funding_org_name  start_date  \
0         Science and Technology Facilities Council  2020-09-30   
1  Directorate for Mathematical & Physical Sciences  2020-09-01   
2                         European Research Council  2020-07-01   
3  Directorate for Mathematical & Physical Sciences  2020-07-01   
4                       Directorate for Engineering  2020-06-01   

  title_language                                              title language  
0             en       The Cosmology of the Early and Late Universe       en  
1             en  Decoding the Infrared Spectra of High Frequenc...       en  
2             en  Overcoming stellar activity in radial velocity...       en  
3             en  CAREER: Theory of Membrane Shape Sensing at th...       en  
4             en  CAREER: Modeling Human Gait to Optimize Exoske...       en

Eg let's see what are the *search fields* available for the *grants* source:

In [306]:
dsl.query("describe schema")['sources']['grants']['search_fields']

['investigators', 'full_data', 'concepts', 'title_only', 'title_abstract_only']

In [307]:
%%dsldf 
search grants 
    in full_data for "graphene AND computer AND iron" 
return grants limit 5

Returned Grants: 5 (total = 10)


funders         active_year  \
0  [{'id': 'grid.454869.2', 'longitude': 37.64049...  [2019, 2020, 2021]   
1  [{'id': 'grid.452899.b', 'longitude': 37.57781...              [2018]   
2  [{'id': 'grid.425823.a', 'longitude': 21.02022...              [2016]   
3  [{'id': 'grid.425823.a', 'longitude': 21.02022...              [2015]   
4  [{'id': 'grid.425823.a', 'longitude': 21.02022...              [2014]   

                                      original_title  start_year    end_date  \
0  Weyl and Dirac semimetals and beyond - predict...        2019  2021-12-31   
1  Проект организации 18-ой Международной конфере...        2018  2018-12-31   
2  Dotacja podmiotowa na utrzymanie potencjału ba...        2016  2016-12-31   
3  Dotacja podmiotowa na utrzymanie potencjału ba...        2015  2015-12-31   
4  Dotacja celowa na prowadzenie w 2014 przez Wyd...        2014  2014-12-31   

         project_num             id                          funding_org_name  \
0        19-43-04129  grant.8413990                Russian Science Foundation   
1        18-02-20097  grant.8731867     Russian Foundation for Basic Research   
2  4491/E-370/S/2016  grant.7397800  Ministry of Science and Higher Education   
3  4491/E-370/S/2015  grant.7397795  Ministry of Science and Higher Education   
4  4491/E-370/M/2014  grant.7397490  Ministry of Science and Higher Education   

   start_date title_language  \
0  2019-01-01             en   
1  2018-01-01             ru   
2  2016-02-22             pl   
3  2015-02-19             pl   
4  2014-04-09             pl   

                                               title language  
0  Weyl and Dirac semimetals and beyond - predict...       en  
1  Project of the organization of the 18th Intern...       ru  
2  Subject subsidy for maintaining the research p...       pl  
3  Subject subsidy for maintaining the research p...       pl  
4  Intentional grant for conducting in 2014 the F...       pl

Special search indexes for persons names permit to perform full text
searches on publications `authors` or grants `investigators`. Please see the
*Researchers Search* section below for more information
on how searches work in this case.

In [308]:
%dsldf search publications in authors for "\"Jennifer A Doudna\"" return publications limit 5

Returned Publications: 5 (total = 315)


year              id                                author_affiliations  \
0  2020  pub.1125959258  [[{'first_name': 'Simon', 'last_name': 'Eitzin...   
1  2020  pub.1126635310  [[{'first_name': 'Innovative Genomics Institut...   
2  2020  pub.1125548532  [[{'first_name': 'Kyle E.', 'last_name': 'Watt...   
3  2020  pub.1125325301  [[{'first_name': 'Ivan E.', 'last_name': 'Ivan...   
4  2020  pub.1125677167  [[{'first_name': 'Michelle F.', 'last_name': '...   

                                               title     type    journal.id  \
0  Machine learning predicts new anti-CRISPR prot...  article  jour.1018982   
1      Blueprint for a Pop-up SARS-CoV-2 Testing Lab  article  jour.1369542   
2  Potent CRISPR-Cas9 inhibitors from Staphylococ...  article  jour.1082971   
3  Cas9 interrogates DNA in discrete steps modula...  article  jour.1082971   
4  Phage-assisted evolution of an adenine base ed...  article  jour.1115214   

                                       journal.title                pages  \
0                             Nucleic Acids Research                  NaN   
1                                            medRxiv  2020.04.11.20061424   
2  Proceedings of the National Academy of Science...            6531-6539   
3  Proceedings of the National Academy of Science...            5853-5860   
4                               Nature Biotechnology                  1-9   

  issue volume  
0   NaN    NaN  
1   NaN    NaN  
2    12    117  
3    11    117  
4   NaN    NaN

### 2.2 `for "search term"`

This optional phrase consists of the keyword `for` followed by a
`search term` `string`, enclosed in double quotes (`"`).

Strings in double quotes can contain nested quotes escaped by a
backslash `\`. This will ensure that the string in nested double quotes
is searched for as if it was a single phrase, not multiple words.

An example of a phrase: `"\"Machine Learning\""` : results must contain
`Machine Learning` as a phrase.

In [309]:
%dsldf search publications for "\"Machine Learning\"" return publications limit 5

Returned Publications: 5 (total = 1105282)


year              id                                author_affiliations  \
0  2020  pub.1124674945  [[{'first_name': 'Tom Arne', 'last_name': 'Ped...   
1  2020  pub.1124670304  [[{'first_name': 'Jon Arne', 'last_name': 'Glo...   
2  2020  pub.1125933988  [[{'first_name': 'Baze University', 'last_name...   
3  2020  pub.1124109981  [[{'first_name': 'AlexanderVE', 'last_name': '...   
4  2020  pub.1124669794  [[{'first_name': 'Zhongyi', 'last_name': 'Sui'...   

                                               title       type  pages issue  \
0  Towards Simulation-based Verification of Auton...    chapter   1-13   NaN   
1  Trustworthy versus Explainable AI in Autonomou...    chapter  37-47   NaN   
2  Solving some variants of vehicle routing probl...    article  63-73     1   
3  Statistics, Data Mining, and Machine Learning ...  monograph    NaN   NaN   
4  Empirical analysis of complex network for mari...    chapter  24-36   NaN   

  volume    journal.id                          journal.title  
0    NaN           NaN                                    NaN  
1    NaN           NaN                                    NaN  
2      4  jour.1317852  Open Journal of Mathematical Sciences  
3    NaN           NaN                                    NaN  
4    NaN           NaN                                    NaN

Example of multiple keywords: `"Machine Learning"` : this searches for
keywords independently.

In [310]:
%dsldf search publications for "Machine Learning" return publications limit 5

Returned Publications: 5 (total = 2339984)


pages  year              id  \
0  297-312  2020  pub.1124034464   
1  201-218  2020  pub.1124034459   
2  333-463  2020  pub.1125380184   
3     1-13  2020  pub.1124674945   
4  507-801  2020  pub.1126070643   

                                 author_affiliations     type  \
0  [[{'first_name': 'Bettina', 'last_name': 'Frit...  chapter   
1  [[{'first_name': 'Susanne', 'last_name': 'Ress...  chapter   
2                                                NaN  chapter   
3  [[{'first_name': 'Tom Arne', 'last_name': 'Ped...  chapter   
4                                                NaN  chapter   

                                               title  
0              „…the trace of the other in the self“  
1                  Positioned struggles over history  
2                                    III. BRĀHMANISM  
3  Towards Simulation-based Verification of Auton...  
4                                     ANHANG. Part 1

Note: Special characters, such as any of `^ " : ~ \ [ ] { } ( ) ! | & +` must be escaped by a backslash `\`. Also, please note escaping rules in
[Python](http://python-reference.readthedocs.io/en/latest/docs/str/escapes.html) (or other languages). For example, when writing a query with escaped quotes, such as `search publications for "\"phrase 1\" AND \"phrase 2\""`, in Python, it is necessary to escape the backslashes as well, so it
would look like: `'search publications for "\\"phrase 1\\" AND \\"phrase 2\\""'`. 

See the [official docs](https://docs.dimensions.ai/dsl/language.html#for-search-term) for more details. 




### 2.3 Boolean Operators

Search term can consist of multiple keywords or phrases connected using
boolean logic operators, e.g. `AND`, `OR` and `NOT`.

In [311]:
%dsldf search publications for "(dose AND concentration)" return publications limit 5

Returned Publications: 5 (total = 5214760)


pages  year              id  \
0    125-158  2020  pub.1125801743   
1    219-238  2020  pub.1124265256   
2      25-49  2020  pub.1125630589   
3    1694347  2020  pub.1124196768   
4  1665-1850  2020  pub.1126070754   

                                 author_affiliations     type  \
0  [[{'first_name': 'Letizia', 'last_name': 'Moni...  chapter   
1  [[{'first_name': 'Samantha', 'last_name': 'Ash...  chapter   
2  [[{'first_name': 'Margaret Clunies', 'last_nam...  chapter   
3  [[{'first_name': 'Anne M', 'last_name': 'de Gr...  article   
4                                                NaN  chapter   

                                               title issue volume  \
0  5. Chemical Alteration and Colour Changes in t...   NaN    NaN   
1  Chapter 11. Political Obligation and the Rule ...   NaN    NaN   
2  The Skald Sagas as a Genre: Definitions and Ty...   NaN    NaN   
3  Effectiveness of a peer-refugee delivered psyc...     1     11   
4  Le socialisme en Allemagne (Fragment du brouil...   NaN    NaN   

     journal.id                           journal.title  
0           NaN                                     NaN  
1           NaN                                     NaN  
2           NaN                                     NaN  
3  jour.1045059  European Journal of Psychotraumatology  
4           NaN                                     NaN

When specifying Boolean operators with keywords such as `AND`, `OR` and
`NOT`, the keywords must appear in all uppercase. 

The operators available are shown in the table below.
.

| Boolean Operator | Alternative Symbol | Description                                                                                                                                                                 |
|------------------|--------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `AND`            | `&&`               | Requires both terms on either side of the Boolean operator to be present for a match.                                                                                       |
| `NOT`            | `!`                | Requires that the following term not be present.                                                                                                                            |
| `OR`             | `||`               | Requires that either term (or both terms) be present for a match.                                                                                                           |
|                  | `+`                | Requires that the following term be present.                                                                                                                                |
|                  | `-`                | Prohibits the following term (that is, matches on fields or documents that do not include that term). The `-` operator is functionally similar to the Boolean operator `!`. |

In [312]:
%dsldf search publications for "(dose OR concentration) AND (-malaria +africa)" return publications limit 5

Returned Publications: 5 (total = 1332804)


year              id                                              title  \
0  2020  pub.1124946803                                   Literature Cited   
1  2020  pub.1125629784  7. The Aryan question: Some general considerat...   
2  2020  pub.1126070451  Heft 4. Exzerpte zur Geschichte Spaniens aus W...   
3  2020  pub.1124248677         12. Culture, Institutions, and Development   
4  2020  pub.1124248673      8. Trade-Related Institutions and Development   

      type    pages                                author_affiliations  
0  chapter  487-564                                                NaN  
1  chapter  177-191  [[{'first_name': 'Arvind', 'last_name': 'Sharm...  
2  chapter  727-852                                                NaN  
3  chapter  414-448  [[{'first_name': 'Gerard', 'last_name': 'Rolan...  
4  chapter  255-307  [[{'first_name': 'Jaime', 'last_name': 'de Mel...

### 2.4 Wildcard Searches

The DSL supports single and multiple character wildcard searches within
single terms. Wildcard characters can be applied to single terms, but
not to search phrases.

In [313]:
%dsldf search publications in title_only for "ital? malaria" return publications limit 5

Returned Publications: 5 (total = 142)


pages  year              id  \
0    1-20  2020  pub.1124231018   
1  101544  2020  pub.1123222257   
2   28-33  2020  pub.1125332077   
3     151  2019  pub.1113815431   
4   34-39  2019  pub.1113201846   

                                 author_affiliations     type  \
0  [[{'first_name': 'Benjamin', 'last_name': 'Rei...  article   
1  [[{'first_name': 'Guido', 'last_name': 'Caller...  article   
2  [[{'first_name': 'Luciana', 'last_name': 'Lepo...  article   
3  [[{'first_name': 'Valentina', 'last_name': 'Ta...  article   
4  [[{'first_name': 'Fiorenza', 'last_name': 'Pan...  article   

                                               title    journal.id  \
0  Seasons in Italy: Northern European travelers,...  jour.1141817   
1  Updated guidelines for malaria prophylaxis in ...  jour.1034401   
2  Clinical management of imported malaria in Ita...  jour.1089291   
3  Investigation on potential malaria vectors (An...  jour.1030597   
4  Increasing imported malaria in children and ad...  jour.1034401   

                            journal.title volume issue  
0  Journal of Tourism and Cultural Change    NaN   NaN  
1  Travel Medicine and Infectious Disease     33   NaN  
2                          Microbiologica     43     1  
3                         Malaria Journal     18     1  
4  Travel Medicine and Infectious Disease     29   NaN

In [314]:
%dsldf search publications in title_only for "it* malaria" return publications limit 5

Returned Publications: 5 (total = 1491)


pages              id  \
0                24  pub.1124106064   
1            109809  pub.1126819455   
2               NaN  pub.1126848860   
3  jbc.ra120.012646  pub.1126501008   
4               271  pub.1126492745   

                                               title volume      type  year  \
0  The effectiveness of older insecticide-treated...     19   article  2020   
1  Modeling pyrethroids repellency and its role o...    136   article  2020   
2  Efficacy of Artemisinin-lumefantrine for the t...    NaN  preprint  2020   
3  Genetic ablation of the mitoribosome in the ma...    NaN   article  2020   
4  Analysis of the Role of TpUB05 Antigen from Th...      9   article  2020   

                                 author_affiliations issue    journal.id  \
0  [[{'first_name': 'Monica P.', 'last_name': 'Sh...     1  jour.1030597   
1  [[{'first_name': 'Berge', 'last_name': 'Tsanou...   NaN  jour.1026215   
2  [[{'first_name': 'Gabriel M.', 'last_name': 'K...   NaN  jour.1380788   
3  [[{'first_name': 'Liqin', 'last_name': 'Ling',...   NaN  jour.1077138   
4  [[{'first_name': 'Jerome Nyhalah', 'last_name'...     4  jour.1047674   

                     journal.title  
0                  Malaria Journal  
1        Chaos Solitons & Fractals  
2                  Research Square  
3  Journal of Biological Chemistry  
4                        Pathogens

| Wildcard Search Type                                             | Special Character | Example                                                                                                                                                                                                                         |
|------------------------------------------------------------------|-------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Single character - matches a single character                    | `?`               | The search string `te?t` would match both `test` and `text`.                                                                                                                                                                    |
| Multiple characters - matches zero or more sequential characters | `*`               | The wildcard search: `tes*` would match `test`, `testing`, and `tester`. You can also use wildcard characters in the middle of a term. For example: `te*t` would match `test` and `text`. `*est` would match `pest` and `test`. |

### 2.5 Proximity Searches

A proximity search looks for terms that are within a specific distance
from one another.

To perform a proximity search, add the tilde character `~` and a numeric
value to the end of a search phrase. For example, to search for a
`formal` and `model` within 10 words of each other in a document, use
the search:

In [315]:
%dsldf search publications for "\"formal model\"~10" return publications limit 5

Returned Publications: 5 (total = 463369)


pages              id                                              title  \
0  189-217  pub.1124248671  6. Institutions, Development, and Growth: Wher...   
1  294-312  pub.1124248742  17. History and the Unanswered Questions of th...   
2  198-220  pub.1124946785  11. Song Traditions in Indigo Buntings: Origin...   
3    42-71  pub.1125789332  2. Trends in Private Sector Grievance Arbitration   
4  595-633  pub.1124248681       16. Institutions, Firm Financing, and Growth   

      type  year                                author_affiliations  
0  chapter  2020  [[{'first_name': 'Steven N.', 'last_name': 'Du...  
1  chapter  2020  [[{'first_name': 'Francis J.', 'last_name': 'G...  
2  chapter  2020  [[{'first_name': 'Robert B.', 'last_name': 'Pa...  
3  chapter  2020  [[{'first_name': 'Dennis R.', 'last_name': 'No...  
4  chapter  2020  [[{'first_name': 'Meghana', 'last_name': 'Ayya...

In [316]:
%dsldf search publications for "\"digital humanities\"~5  +ontology" return publications limit 5

Returned Publications: 5 (total = 6617)


year              id                                author_affiliations  \
0  2020  pub.1124427860  [[{'first_name': 'Vincent', 'last_name': 'Jail...   
1  2020  pub.1125401563  [[{'first_name': 'Luis A.', 'last_name': 'Pine...   
2  2020  pub.1126922287                                                NaN   
3  2020  pub.1126879229  [[{'first_name': 'Alba', 'last_name': 'Silva',...   
4  2020  pub.1126879230  [[{'first_name': 'Xosé', 'last_name': 'López-G...   

                                               title       type   pages  \
0  Describing, comparing and analysing digital ur...    article  e00135   
1  Practical non-monotonic knowledge-base system ...    article  102214   
2                  Transformative Digital Humanities  monograph     NaN   
3                  Mobile Devices and Mobile Content    chapter   35-50   
4         New Narratives in the Age of Visualization    chapter   51-63   

  volume    journal.id                                      journal.title  \
0     17  jour.1151279  Digital Applications in Archaeology and Cultur...   
1     57  jour.1121462                Information Processing & Management   
2    NaN           NaN                                                NaN   
3    NaN           NaN                                                NaN   
4    NaN           NaN                                                NaN   

  issue  
0   NaN  
1     3  
2   NaN  
3   NaN  
4   NaN

The distance referred to here is the number of term movements needed to match the specified phrase.  
In the example above, if `formal` and `model` were 10 spaces apart in a
field, but `formal` appeared before `model`, more than 10 term movements
would be required to move the terms together and position `formal` to
the right of `model` with a space in between.

### 2.5 Term Boosting

The DSL provides the relevance level of matching documents based on the
terms found. To boost a term use the caret symbol $^$ with a boost
factor (a number) at the end of the term you are searching. The higher
the boost factor, the more relevant the term will be.

Boosting allows the user to control the relevance of a document by
boosting its term. For example, when searching for $"formal model"$ and
you want the term "formal" to be more relevant, you can boost it by
adding the $^$ symbol along with the boost factor immediately after the
term. For example, you could type:

In [317]:
%dsldf search publications for "formal^4 model" return publications limit 5 

Returned Publications: 5 (total = 4351279)


year              id                                author_affiliations  \
0  2020  pub.1122051578  [[{'first_name': 'Andrea', 'last_name': 'Crow'...   
1  2020  pub.1123020742  [[{'first_name': 'Jelena', 'last_name': 'Rafai...   
2  2020  pub.1124487011  [[{'first_name': 'Olivier', 'last_name': 'Roy'...   
3  2020  pub.1124034442  [[{'first_name': 'Achim', 'last_name': 'Oberg'...   
4  2020  pub.1124034535  [[{'first_name': 'Thomas', 'last_name': 'Hilge...   

                                               title     type    pages  
0               CHAPTER X. Mobilizing Academic Labor  chapter  192-205  
1    An Economic Survey of the Kingdom of Yugoslavia  chapter    35-56  
2        7. The Failure of Political Islam Revisited  chapter  167-180  
3                                          Netzwerke  chapter  191-218  
4  Helden, Freaks und dunkle Ritter: Batman in Ho...  chapter  177-214

This will make documents with the term $formal$ appear more relevant.
You can also boost Phrase Terms as in the example:

In [318]:
%dsldf search publications for "\"formal model\"^4 \"Lindenmayer system\"" return publications limit 5  

Returned Publications: 5 (total = 1149)


pages issue  year              id  \
0      54-65     1  2020  pub.1120164618   
1        NaN   NaN  2020  pub.1113605618   
2        NaN   NaN  2020  pub.1126386421   
3  1615-1627     9  2019  pub.1117308212   
4    107-131   1-2  2019  pub.1113061099   

                                 author_affiliations     type  \
0  [[{'first_name': 'Brenden M.', 'last_name': 'L...  article   
1                                                NaN     book   
2                                                NaN     book   
3  [[{'first_name': 'Cédric', 'last_name': 'Gauch...  article   
4  [[{'first_name': 'José L.', 'last_name': 'Besa...  article   

                                               title volume    journal.id  \
0  People Infer Recursive Visual Concepts from Ju...      3  jour.1320442   
1  From Astrophysics to Unconventional Computatio...     35           NaN   
2  Applications of Evolutionary Computation, 23rd...  12104           NaN   
3                                   GAUCHEREL et al.     10  jour.1044080   
4  Math and Music, Models and Metaphors: Alberto ...     38  jour.1138316   

                      journal.title  
0    Computational Brain & Behavior  
1                               NaN  
2                               NaN  
3  Methods in Ecology and Evolution  
4         Contemporary Music Review

By default, the boost factor is 1. Although the boost factor must be
positive, it can be less than 1 (for example, it could be 0.2).

## 3. Field Searching

Field searching allows to use a specific `field` of a `source` as a
query filter. For example, this can be a
[Literal](supported-types.ipynb) field such as the $type$ of a
publication, its $date$, $mesh terms$, etc.. Or it can be an
[entity](data-entities.ipynb) field, such as the $journal title$ for a
publication, the $country name$ of its author affiliations, etc..

**What are the fields available for each source?** See the [data sources](https://docs.dimensions.ai/dsl/data-sources.html) section of the documentation. 

Alternatively, we can use the 'schema' API ([describe](https://docs.dimensions.ai/dsl/data-sources.html#metadata-api)) to return this information programmatically: 

In [319]:
%dsldocs publications  

sources                       field           type  \
0   publications                   altmetric          float   
1   publications                altmetric_id        integer   
2   publications                     authors           json   
3   publications                    book_doi         string   
4   publications           book_series_title         string   
5   publications                  book_title         string   
6   publications                category_bra     categories   
7   publications                category_for     categories   
8   publications                category_hra     categories   
9   publications            category_hrcs_hc     categories   
10  publications           category_hrcs_rac     categories   
11  publications           category_icrp_cso     categories   
12  publications            category_icrp_ct     categories   
13  publications               category_rcdc     categories   
14  publications                category_sdg     categories   
15  publications                category_uoa     categories   
16  publications                    concepts         string   
17  publications                        date           date   
18  publications               date_inserted           date   
19  publications                         doi         string   
20  publications        field_citation_ratio          float   
21  publications            funder_countries      countries   
22  publications                     funders  organizations   
23  publications                          id         string   
24  publications                        issn         string   
25  publications                       issue         string   
26  publications                     journal       journals   
27  publications               journal_lists         string   
28  publications                     linkout         string   
29  publications                  mesh_terms         string   
30  publications      open_access_categories    open_access   
31  publications                       pages         string   
32  publications                       pmcid         string   
33  publications                        pmid         string   
34  publications           proceedings_title         string   
35  publications                   publisher         string   
36  publications            recent_citations        integer   
37  publications               reference_ids         string   
38  publications     relative_citation_ratio          float   
39  publications         research_org_cities         cities   
40  publications      research_org_countries      countries   
41  publications  research_org_country_names         string   
42  publications    research_org_state_codes         states   
43  publications    research_org_state_names         string   
44  publications               research_orgs  organizations   
45  publications                 researchers    researchers   
46  publications   resulting_publication_doi         string   
47  publications        supporting_grant_ids         string   
48  publications                 times_cited        integer   
49  publications                       title         string   
50  publications                        type         string   
51  publications                      volume         string   
52  publications                        year        integer   

                                          description  is_filter  is_entity  \
0                          Altmetric attention score.       True      False   
1                            AltMetric Publication ID       True      False   
2   Ordered list of authors names and their affili...       True      False   
3   The DOI of the book a chapter belongs to (note...       True      False   
4       The title of the book series book, belong to.      False      False   
5   The title of the book a chapter belongs to (no...      False      False   
6   `Broad Research Areas <https://app.dimensions....

### 3.1 `where`

This optional phrase consists of the keyword `where` followed by a
`filters` phrase consisting of DSL filter expressions, as described
below.

In [320]:
%dsldf search publications where type = "book" return publications limit 5

Returned Publications: 5 (total = 286539)


id                                              title  type  \
0  pub.1124034527             Hollywood im Zeitalter des Post Cinema  book   
1  pub.1124034601                         Wagner - Weimar - Eisenach  book   
2  pub.1124946767  Ecology and Evolution of Acoustic Communicatio...  book   
3  pub.1124907235            Weibliche Herrschaft im 18. Jahrhundert  book   
4  pub.1125300612            Contents Tourism and Pop Culture Fandom  book   

   year  
0  2020  
1  2020  
2  2020  
3  2020  
4  2020

If a `for` phrase is also used in a filtered query, the
system will first apply the filters, and then search the resulting
restricted set of documents for the `search term`.

In [321]:
%dsldf search publications for "malaria" where type = "book" return publications limit 5

Returned Publications: 5 (total = 12194)


year              id  type  \
0  2020  pub.1126879214  book   
1  2020  pub.1126251587  book   
2  2020  pub.1126499453  book   
3  2020  pub.1126265096  book   
4  2020  pub.1126251608  book   

                                               title  
0  The Routledge History Handbook of Central and ...  
1              21st Century Nanoscience – A Handbook  
2                                          Iteration  
3  Médecins Sans Frontières and Humanitarian Situ...  
4  Microbial Mitigation of Stress Response of Foo...

### 3.2 `in`

For convenience, the DSL also supports shorthand notation for filters
where a particular field should be restricted to a specified range or
list of values (although the same logic may be expressed using complex
filters as shown below).

Syntax: a **range filter** consists of the `field` name, the keyword `in`, and a
range of values enclosed in square brackets (`[]`), where the range
consists of a `low` value, colon `:`, and a `high` value.

In [322]:
%%dsldf 
search grants 
    for "malaria" 
    where start_year in [ 2010 : 2015 ] 
return grants limit 5

Returned Grants: 5 (total = 2965)


id  start_date  project_num    end_date  start_year  \
0  grant.4729738  2015-12-28  R21AI120981  2017-11-30        2015   
1  grant.4729736  2015-12-24  R21AI120973  2019-02-28        2015   
2  grant.4729699  2015-12-21  R21AI109439  2018-11-30        2015   
3  grant.4854433  2015-12-18        91488  2018-12-18        2015   
4  grant.8821176  2015-12-10   MIS-311250  2019-09-30        2015   

  title_language                                              title  \
0             en  Bloodborne tropical pathogen detection using m...   
1             en  Field-deployable Assay for Differential Diagno...   
2             en  T cell driven antigen discovery for vaccine ca...   
3             en  Senior Fellowship for Dr. Eduardo Samo Gudo: E...   
4             en  Biology, Ecology & Management of Emerging Dise...   

                                      original_title  \
0  Bloodborne tropical pathogen detection using m...   
1  Field-deployable Assay for Differential Diagno...   
2  T cell driven antigen discovery for vaccine ca...   
3  Senior Fellowship for Dr. Eduardo Samo Gudo: E...   
4  Biology, Ecology & Management of Emerging Dise...   

                                             funders language  \
0  [{'id': 'grid.419681.3', 'state_name': 'Maryla...       en   
1  [{'id': 'grid.419681.3', 'state_name': 'Maryla...       en   
2  [{'id': 'grid.419681.3', 'state_name': 'Maryla...       en   
3  [{'id': 'grid.452969.5', 'linkout': ['https://...       en   
4  [{'id': 'grid.482914.2', 'state_name': 'Distri...       en   

                      active_year  \
0              [2015, 2016, 2017]   
1  [2015, 2016, 2017, 2018, 2019]   
2        [2015, 2016, 2017, 2018]   
3        [2015, 2016, 2017, 2018]   
4  [2015, 2016, 2017, 2018, 2019]   

                                    funding_org_name  
0  National Institute of Allergy and Infectious D...  
1  National Institute of Allergy and Infectious D...  
2  National Institute of Allergy and Infectious D...  
3                              Volkswagen Foundation  
4         National Institute of Food and Agriculture

Syntax: a **list filter** consists of the `field` name, the keyword `in`, and a list
of one or more `value` s enclosed in square brackets (`[]`), where
values are separated by commas (`,`):

In [323]:
%%dsldf 
search grants 
    for "malaria" 
    where research_org_name in [ "UC Berkeley", "UC Davis", "UCLA"  ] 
return grants limit 5

Returned Grants: 0
WARNINGS [1]
Field 'research_org_name' is deprecated in favor of research_orgs. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


Empty DataFrame
Columns: []
Index: []

### 3.3 `count` - filter function

The filter function `count` is supported on some fields in
[publications](publications.ipynb) (e.g. `researchers` and
`research_orgs`).

Use of this filter is shown on the example below:

In [324]:
%%dsldf 
search publications 
    for "malaria" 
    where count(research_orgs) > 5 
return research_orgs limit 5

Returned Research_orgs: 5


id  count     state_name  city_name   latitude  \
0    grid.4991.5   1445    Oxfordshire     Oxford  51.753437   
1    grid.8991.9   1374         Camden     London  51.520900   
2  grid.38142.3c    976  Massachusetts  Cambridge  42.377052   
3  grid.21107.35    802       Maryland  Baltimore  39.328888   
4    grid.7445.2    721    Westminster     London  51.498600   

                                           name        types    country_name  \
0                          University of Oxford  [Education]  United Kingdom   
1  London School of Hygiene & Tropical Medicine  [Education]  United Kingdom   
2                            Harvard University  [Education]   United States   
3                      Johns Hopkins University  [Education]   United States   
4                       Imperial College London  [Education]  United Kingdom   

                        linkout  longitude acronym  
0        [http://www.ox.ac.uk/]  -1.254010     NaN  
1     [http://www.lshtm.ac.uk/]  -0.130700   LSHTM  
2     [http://www.harvard.edu/] -71.116650     NaN  
3        [https://www.jhu.edu/] -76.620280     JHU  
4  [http://www.imperial.ac.uk/]  -0.175478     NaN

Number of publications with more than 50 researcher.

In [325]:
%%dsldf 
search publications 
    for "malaria" 
    where count(researchers) > 50 
return publications limit 5

Returned Publications: 5 (total = 173)


pages  year              id  \
0       1-14  2020  pub.1126151286   
1  1345-1360  2020  pub.1125560167   
2      60-79  2020  pub.1122257132   
3  1749-1768  2019  pub.1121303277   
4  1457-1973  2019  pub.1121951380   

                                 author_affiliations     type  \
0  [[{'first_name': 'Drahomíra', 'last_name': 'Fa...  article   
1  [[{'first_name': 'Rob W', 'last_name': 'van de...  article   
2  [[{'first_name': 'GBD 2017 Lower Respiratory I...  article   
3  [[{'first_name': 'Christina', 'last_name': 'Fi...  article   
4  [[{'first_name': 'Andrea', 'last_name': 'Cossa...  article   

                                               title    journal.id  \
0  Genetic tool development in marine protists: e...  jour.1033763   
1  Triple artemisinin-based combination therapies...  jour.1077219   
2  Quantifying risks and interventions that have ...  jour.1030033   
3  Global, Regional, and National Cancer Incidenc...  jour.1051466   
4  Guidelines for the use of flow cytometry and c...  jour.1054998   

                    journal.title  issue volume  
0                  Nature Methods    NaN    NaN  
1                      The Lancet  10233    395  
2  The Lancet Infectious Diseases      1     20  
3                   JAMA Oncology     12      5  
4  European Journal of Immunology     10     49

Number of publications with more than one researcher.

In [326]:
%%dsldf 
search publications
where count(researchers) > 1
return funders limit 5

Returned Funders: 5


id    count   longitude  \
0  grid.419696.5  1711981  116.339830   
1  grid.270680.b   629227    4.363670   
2  grid.48336.3a   551777  -77.101190   
3  grid.424020.0   550809  116.316284   
4  grid.54432.34   518605  139.740390   

                                     linkout         types  \
0  [http://www.nsfc.gov.cn/publish/portal1/]  [Government]   
1         [http://ec.europa.eu/index_en.htm]  [Government]   
2                   [http://www.cancer.gov/]  [Government]   
3              [http://www.most.gov.cn/eng/]  [Government]   
4                   [http://www.jsps.go.jp/]   [Nonprofit]   

                                                name   country_name  \
0       National Natural Science Foundation of China          China   
1                                European Commission        Belgium   
2                          National Cancer Institute  United States   
3  Ministry of Science and Technology of the Peop...          China   
4         Japan Society for the Promotion of Science          Japan   

   city_name acronym   latitude state_name  
0    Beijing    NSFC  40.005177        NaN  
1   Brussels      EC  50.851650        NaN  
2  Rockville     NCI  39.004326   Maryland  
3    Beijing    MOST  39.827835        NaN  
4      Tokyo    JSPS  35.687160        NaN

International collaborations: number of publications with more than one author and affiliations located in more than one country.

In [327]:
%%dsldf 
search publications
where count(researchers) > 1
and count(research_org_countries) > 1
return funders limit 5

Returned Funders: 5


id   count city_name   latitude  \
0  grid.419696.5  416514   Beijing  40.005177   
1  grid.270680.b  321979  Brussels  50.851650   
2  grid.424150.6  147782      Bonn  50.699340   
3  grid.424020.0  138046   Beijing  39.827835   
4  grid.54432.34  130524     Tokyo  35.687160   

                                                name acronym         types  \
0       National Natural Science Foundation of China    NSFC  [Government]   
1                                European Commission      EC  [Government]   
2                         German Research Foundation     DFG    [Facility]   
3  Ministry of Science and Technology of the Peop...    MOST  [Government]   
4         Japan Society for the Promotion of Science    JSPS   [Nonprofit]   

  country_name                                    linkout   longitude  
0        China  [http://www.nsfc.gov.cn/publish/portal1/]  116.339830  
1      Belgium         [http://ec.europa.eu/index_en.htm]    4.363670  
2      Germany                    [http://www.dfg.de/en/]    7.147797  
3        China              [http://www.most.gov.cn/eng/]  116.316284  
4        Japan                   [http://www.jsps.go.jp/]  139.740390

Domestic collaborations: number of publications with more than one author and more than one affiliation located in exactly one country.

In [328]:
%%dsldf 
search publications
where count(researchers) > 1
and count(research_org_countries) = 1
return funders limit 5

Returned Funders: 5


id    count  city_name   latitude  \
0  grid.419696.5  1254361    Beijing  40.005177   
1  grid.48336.3a   404524  Rockville  39.004326   
2  grid.424020.0   401857    Beijing  39.827835   
3  grid.54432.34   355863      Tokyo  35.687160   
4  grid.280785.0   312686   Bethesda  38.997833   

                                                name acronym         types  \
0       National Natural Science Foundation of China    NSFC  [Government]   
1                          National Cancer Institute     NCI  [Government]   
2  Ministry of Science and Technology of the Peop...    MOST  [Government]   
3         Japan Society for the Promotion of Science    JSPS   [Nonprofit]   
4     National Institute of General Medical Sciences   NIGMS    [Facility]   

    country_name                                        linkout   longitude  \
0          China      [http://www.nsfc.gov.cn/publish/portal1/]  116.339830   
1  United States                       [http://www.cancer.gov/]  -77.101190   
2          China                  [http://www.most.gov.cn/eng/]  116.316284   
3          Japan                       [http://www.jsps.go.jp/]  139.740390   
4  United States  [http://www.nigms.nih.gov/Pages/default.aspx]  -77.099380   

  state_name  
0        NaN  
1   Maryland  
2        NaN  
3        NaN  
4   Maryland

### 3.4 Filter Operators

A simple filter expression consists of a `field` name, an in-/equality
operator `op`, and the desired field `value`. 

The `value` must be a
`string` enclosed in double quotes (`"`) or an integer (e.g. `1234`).

The available operators are:

| `op`           | meaning                                                                                  |
|----------------|------------------------------------------------------------------------------------------|
| `=`            | *is* (or *contains* if the given `field` is multi-value)                                 |
| `!=`           | *is not*                                                                                 |
| `>`            | *is greater than*                                                                        |
| `<`            | *is less than*                                                                           |
| `>=`           | *is greater than or equal to*                                                            |
| `<=`           | *is less than or equal to*                                                               |
| `~`            | *partially matches* (see partial-string-matching below) |
| `is empty`     | *is empty* (see emptiness-filters below)                      |
| `is not empty` | *is not empty* (see emptiness-filters below)                  |

A couple of examples 

In [329]:
%dsldf search datasets where year > 2010 and year < 2012 return datasets limit 5

Returned Datasets: 5 (total = 38337)


id                                              title  \
0  10993892  (Table 1) Radiocarbon ages of samples taken fr...   
1  10993247  Average fluorescence and dissolved iron and Fe...   
2  10993244  (Table 1) Average fluorescence in the surface ...   
3  10993241  Dissolved and dissolvable iron concentrations ...   
4  10993193  (Table 1) Movement parameters of nine adult fe...   

                                             authors  year   keywords  \
0  [{'name': 'Minna Väliranta', 'orcid': ''}, {'n...  2011  [PANGAEA]   
1  [{'name': 'Charles-Edouard Thuróczy', 'orcid':...  2011  [PANGAEA]   
2  [{'name': 'Charles-Edouard Thuróczy', 'orcid':...  2011  [PANGAEA]   
3  [{'name': 'Charles-Edouard Thuróczy', 'orcid':...  2011  [PANGAEA]   
4  [{'name': 'Jean-François Therrien', 'orcid': '...  2011  [PANGAEA]   

     journal.id                                      journal.title  
0  jour.1020344                            Journal of Biogeography  
1  jour.1023157  Deep Sea Research Part II Topical Studies in O...  
2           NaN                                                NaN  
3  jour.1312079                    Journal of Geophysical Research  
4  jour.1023041                           Journal of Avian Biology

In [330]:
%dsldf search patents where assignees != "grid.410484.d" return patents limit 5

Returned Patents: 5 (total = 39704493)


assignees filing_status  \
0  [{'id': 'grid.6584.f', 'name': 'Robert Bosch (...         Grant   
1                                                NaN   Application   
2  [{'id': 'grid.418190.5', 'name': 'Thermo Fishe...         Grant   
3  [{'id': 'grid.471210.1', 'name': 'Kuraray (Jap...         Grant   
4  [{'id': 'grid.471143.4', 'name': 'Fujikura (Ja...         Grant   

  publication_date  year                                     inventor_names  \
0       2009-12-09  2001           [TUMBACK, STEFAN, SCHNELLE, KLAUS-PETER]   
1       2009-12-10  2009                                      [SHKEDI, ROY]   
2       2009-12-09  1996               [RIVIELLO, JOHN, M., REY, MARIA, A.]   
3       2009-12-09  1998  [TANAKA, EIJI, HIGASHI, TAMIO, KITAMURA, TAKAN...   
4       2009-12-09  1997  [NAKAI, MICHIHIRO, SHIMA, KENSUKE, HIDAKA, HIR...   

                 id  granted_year  times_cited  \
0     EP-1409282-B1        2009.0            0   
1  WO-2009149128-A2           NaN            1   
2     EP-0868664-B1        2009.0            0   
3     EP-0861808-B1        2009.0            1   
4     EP-0805365-B1        2009.0            0   

                           assignee_names  \
0  [Robert Bosch GmbH, BOSCH GMBH ROBERT]   
1                            [SHKEDI ROY]   
2              [Dionex Corp, DIONEX CORP]   
3            [Kuraray Co Ltd, KURARAY CO]   
4            [Fujikura Ltd, FUJIKURA LTD]   

                                               title  
0  METHODS FOR OPERATING A MOTOR VEHICLE DRIVEN B...  
1  TARGETED TELEVISION ADVERTISEMENTS ASSOCIATED ...  
2  MULTI-CYCLE LOOP INJECTION FOR TRACE ANALYSIS ...  
3                    Waste water treatment apparatus  
4  Optical waveguide grating and production metho...

### 3.5 Partial string matching with `~`

The `~` operator indicates that the given `field` need only partially,
instead of exactly, match the given `string` (the `value` used with this
operator must be a `string`, not an integer).

For example, the filter `where research_orgs.name~"Saarland Uni"` would
match both the organization named "Saarland University" and the one
named "Universitätsklinikum des Saarlandes", and any other organization
whose name includes the terms "Saarland" and "Uni" (the order is
unimportant). 

In [366]:
%%dsldf 
search patents 
    where assignee_names ~ "IBM" 
return assignees limit 5

Returned Assignees: 5


id   count      city_name                              name  \
0  grid.410484.d  329418         Armonk               IBM (United States)   
1  grid.471366.1   22089    George Town  GlobalFoundries (Cayman Islands)   
2  grid.14648.3f    5071     Winchester              IBM (United Kingdom)   
3  grid.420451.6    3555  Mountain View                            Google   
4  grid.472772.3    2717        Beijing                    Lenovo (China)   

     country_name  
0   United States  
1  Cayman Islands  
2  United Kingdom  
3   United States  
4           China

### 3.6 Emptiness filters `is empty`

To filter records which contain specific field or to filter those which
contain an empty field, it is possible to use something like
`where research_orgs is not empty` or `where issn is empty`.

In [367]:
%%dsldf
search publications 
    for "iron graphene" 
    where researchers is empty 
    and research_orgs is not empty 
return publications[id+title+researchers+research_orgs+type] limit 5

Returned Publications: 5 (total = 1951)


research_orgs  \
0  [{'id': 'grid.412030.4', 'city_name': 'Tianjin...   
1  [{'id': 'grid.6734.6', 'state_name': 'Berlin',...   
2  [{'id': 'grid.8547.e', 'state_name': 'Shanghai...   
3  [{'id': 'grid.411510.0', 'city_name': 'Xuzhou'...   
4  [{'id': 'grid.33764.35', 'state_name': 'Heilon...   

                                               title     type              id  
0  Facile Approach to Prepare rGO@Fe3O4 Microsphe...  article  pub.1126764136  
1  Specific adsorption sites and conditions deriv...  article  pub.1126829056  
2  Adsorptive removal of tetracycline by sustaina...  article  pub.1124956822  
3  Sulfur-Doped Alkylated Graphene Oxide as High-...  article  pub.1124438091  
4  Molecular Dynamics Simulations of Melting Iron...  article  pub.1125095130

## 4. Searching for Researchers

The DSL offers different mechanisms for searching for researchers (e.g.
publication authors, grant investigators), each of them presenting
specific advantages.

### 4.1 Exact name searches

Special full-text indices allows to look up a researcher's name and
surname **exactly as they appear in the source documents** they derive from.

This approach has a broad scope, as it allows to search the full
collection of Dimensions documents irrespectively of whether a
researcher was succesfully disambiguated (and hence given a Dimensions
ID). On the other hand, this approach will only match names as they
appear in the source document, so different spellings or initials are
not necessarily returned via a single query. 

```
search in [authors|investigators|inventors]
```

It is possible to look up publications authors using a specific
`search index` called `authors`. 

This method expects case insensitive
phrases, in format $"<first name> <last name>"$ or reverse order. Note
that strings in double quotes that contain nested quotes must always be
escaped by a backslash `\`.

In [368]:
%dsldf search publications in authors for "\"Charles Peirce\"" return publications limit 5

Returned Publications: 5 (total = 195)


pages              id                                 title     type  \
0  202-237  pub.1110025899                     The Law of Mind 1  chapter   
1      1-3  pub.1106516825       Proem The Rules of Philosophy 1  chapter   
2  157-178  pub.1110025908        The Architecture of Theories 1  chapter   
3  106-130  pub.1110025906                 The Order of Nature 1  chapter   
4  179-201  pub.1110025909  The Doctrine of Necessity Examined 1  chapter   

   year                                author_affiliations  
0  2017  [[{'first_name': 'Charles S', 'last_name': 'Pe...  
1  2017  [[{'first_name': 'Charles S', 'last_name': 'Pe...  
2  2017  [[{'first_name': 'Charles S', 'last_name': 'Pe...  
3  2017  [[{'first_name': 'Charles S', 'last_name': 'Pe...  
4  2017  [[{'first_name': 'Charles S', 'last_name': 'Pe...

Instead of first name, initials can also be used. These are examples of
valid research search phrases:

-   `\"Peirce, Charles S.\"`
-   `\"Charles S. Peirce\"`
-   `\"CS Peirce\"`
-   `\"Peirce CS\"`
-   `\"C S Peirce\"`
-   `\"Peirce C S\"`
-   `\"C Peirce\"`
-   `\"Peirce C\"`
-   `\"Charles Peirce\"`
-   `\"Peirce Charles\"`

**Warning**: In order to produce valid results an author or an investigator search
query must contain **at least two components or more** (e.g., name and
surname, either in full or initials).

Investigators search is similar to *authors* search, only it allows to search on `grants` and
`clinical trials` using a separate search index `investigators`, and on
`patents` using the index `inventors`.

In [369]:
%%dsldf 
search clinical_trials in investigators for "\"John Smith\"" 
return clinical_trials limit 5

Returned Clinical_trials: 2 (total = 2)


title           id  \
0  VEPTR Implantation to Treat Children With Earl...  NCT00689533   
1  Prospective Evaluation of Symptom Resolution i...  NCT01241149   

                                investigator_details  \
0  [[John M Flynn, MD, Principal Investigator, Ch...   
1  [[Ellie Mentler, MD, Principal Investigator, U...   

                                        active_years  
0  [2008, 2009, 2010, 2011, 2012, 2013, 2014, 201...  
1                                                NaN

In [370]:
%%dsldf 
search grants in investigators for "\"Satoko Shimazaki\"" 
return grants limit 5

Returned Grants: 4 (total = 4)


id  start_date    project_num    end_date  start_year  \
0  grant.7925589  2020-09-01  FEL-263245-19  2021-08-31        2020   
1  grant.7527261  2018-04-01       18K00431  2021-03-31        2018   
2  grant.5858713  2015-04-01       15K02313  2018-03-31        2015   
3  grant.6086985  2012-04-01       24520310  2015-03-31        2012   

  title_language                                              title  \
0             en  Kabuki Actors, Print Technology, and the Theat...   
1             ja  Genealogy research on female saints in the Pal...   
2             en  Images of Women in the Old English Lives of Sa...   
3             en  Reception and Transfromation of the Images of ...   

                                      original_title  \
0  Kabuki Actors, Print Technology, and the Theat...   
1           古・中英語期における女性聖人伝の系譜研究：Aelfricのテクストと言語を中心に   
2  Images of Women in the Old English Lives of Sa...   
3  Reception and Transfromation of the Images of ...   

                                             funders language  \
0  [{'id': 'grid.422239.c', 'state_name': 'Distri...       en   
1  [{'id': 'grid.54432.34', 'linkout': ['http://w...       ja   
2  [{'id': 'grid.54432.34', 'linkout': ['http://w...       en   
3  [{'id': 'grid.54432.34', 'linkout': ['http://w...       en   

                active_year                            funding_org_name  
0              [2020, 2021]       National Endowment for the Humanities  
1  [2018, 2019, 2020, 2021]  Japan Society for the Promotion of Science  
2  [2015, 2016, 2017, 2018]  Japan Society for the Promotion of Science  
3  [2012, 2013, 2014, 2015]  Japan Society for the Promotion of Science

In [371]:
%%dsldf 
search patents in inventors for "\"John Smith\"" 
return patents limit 5

Returned Patents: 5 (total = 501)


id  times_cited  \
0  US-20020160362-A1            0   
1       GB-2384299-B            0   
2  US-20050133900-A1            2   
3    IE-S20030195-A2            0   
4       GB-2513101-A            0   

                                      assignee_names  \
0                 [AstraZeneca AB, SMITH JOHN CRAIG]   
1  [Llanelli Radiators Ltd, Calsonic Kansei UK Lt...   
2                         [Tessera Inc, TESSERA INC]   
3                                       [SMITH JOHN]   
4                               [Eley Ltd, ELEY LTD]   

                                               title publication_date  \
0                                  Diagnostic method       2002-10-31   
1                          Automotive heat exchanger       2006-03-22   
2  Microelectronic assemblies with composite cond...       2005-06-23   
3  A lockable safety insert for an electrical dom...       2004-11-03   
4                               Ammunition cartridge       2014-10-22   

  inventor_names filing_status  year  \
0   [John Smith]   Application  2001   
1   [SMITH JOHN]         Grant  2002   
2   [John Smith]   Application  2005   
3   [SMITH JOHN]         Grant  2003   
4   [SMITH JOHN]   Application  2013   

                                           assignees  granted_year  
0  [{'id': 'grid.418151.8', 'country_name': 'Swed...           NaN  
1  [{'id': 'grid.472810.8', 'country_name': 'Unit...        2006.0  
2  [{'id': 'grid.455499.0', 'country_name': 'Unit...           NaN  
3                                                NaN        2004.0  
4                                                NaN           NaN

### 4.2 Fuzzy Searches

This type of search is similar to *full-text
search*, with the difference that it
allows searching by only a part of a name, e.g. only the 'last name' of
a person, by using the `where` clause. 

**Note** At this moment, this type of search is only available for
`publications`. Other sources will add this option in the future.

For example:

In [372]:
%%dsldf 
search publications where authors = "Hawking" 
return publications limit 5[id+doi+title+authors] limit 10

Returned Errors: 1
1 QuerySyntaxError found
1 ParserError found
  * [Line 1:73] ('[') mismatched input '[' expecting <EOF>


Generally speaking, using a `where` clause to search authors is less
precise that using the relevant exact-search syntax. 

On the other hand, using a
`where` clause can be handy if one wants to **combine an author search
with another full-text search index**.

For example:

In [373]:
%%dsldf 
search publications 
    in title_abstract_only for "dna replication" 
    where authors = "smith"  
return publications limit 5

Returned Publications: 5 (total = 1511)


pages              id                                              title  \
0    11  pub.1124060243  Longitudinal epigenome-wide association studie...   
1    46  pub.1125664041  An epigenome-wide association study of posttra...   
2    37  pub.1124910780  Genetic associations with clozapine-induced my...   
3   NaN  pub.1126888400  Telomere Length and Autism Spectrum Disorder W...   
4  1760  pub.1126670480  Biophysical Screens Identify Fragments That Bi...   

  volume     type  year                                author_affiliations  \
0     12  article  2020  [[{'first_name': 'Clara', 'last_name': 'Snijde...   
1     12  article  2020  [[{'first_name': 'Mark W.', 'last_name': 'Logu...   
2     10  article  2020  [[{'first_name': 'Paul', 'last_name': 'Lacaze'...   
3    NaN  article  2020  [[{'first_name': 'Candace R.', 'last_name': 'L...   
4     25  article  2020  [[{'first_name': 'Troy E.', 'last_name': 'Mess...   

  issue    journal.id             journal.title  
0     1  jour.1042271      Clinical Epigenetics  
1     1  jour.1042271      Clinical Epigenetics  
2     1  jour.1045271  Translational Psychiatry  
3   NaN  jour.1039397           Autism Research  
4     7  jour.1312072                 Molecules

### 4.3 Using the disambiguated Researchers database

The Dimensions [Researchers](https://docs.dimensions.ai/dsl/datasource-researchers.html) source is a database of
researchers information algorithmically extracted and disambiguated from
all of the other content sources (publications, grants, clinical trials
etc..).

By using the `researchers` source it is possible to match an
'aggregated' person object linking together multiple publication
authors, grant investigators etc.. irrespectively of the form their
names can take in the original source documents.

However, since database does not contain all authors and investigators information
available in Dimensions. 

E.g. think of authors from older publications,
or authors with very common names that are difficult to disambiguate, or
very new authors, who have only one or few publications. In such cases,
using full-text authors search might be more
appropriate.

Examples:

In [374]:
%%dsldf 
search researchers for "\"Satoko Shimazaki\"" 
return researchers[basics+obsolete] 

Returned Researchers: 4 (total = 4)


last_name first_name                                      research_orgs  \
0  Shimazaki     Satoko  [{'id': 'grid.19006.3e', 'state_name': 'Califo...   
1  Shimazaki     Satoko  [{'id': 'grid.266190.a', 'state_name': 'Colora...   
2  Shimazaki     Satoko                                                NaN   
3  Shimazaki     Satoko                                                NaN   

                   id  obsolete  
0  ur.014307627665.09       NaN  
1  ur.015527473602.63       NaN  
2  ur.010537333602.30       1.0  
3   ur.07751146721.59       NaN

NOTE pay attentiont to the `obsolete` field. This indicates the researcher ID status. 0 means that the researcher ID is still **active**, 1 means that the researcher ID is **no longer valid**. This is due to the ongoing process of refinement of Dimensions researchers. 

Hence the query above is best written like this:

In [375]:
%%dsldf 
search researchers where obsolete=0 for "\"Satoko Shimazaki\"" 
return researchers[basics+obsolete] 

Returned Researchers: 3 (total = 3)


research_orgs                  id  \
0  [{'id': 'grid.266190.a', 'longitude': -105.265...  ur.015527473602.63   
1  [{'id': 'grid.19006.3e', 'longitude': -118.444...  ur.014307627665.09   
2                                                NaN   ur.07751146721.59   

   last_name first_name  
0  Shimazaki     Satoko  
1  Shimazaki     Satoko  
2  Shimazaki     Satoko

With `Researchers`, one can use other fields as well:

In [403]:
%%dsldf 
search researchers 
    where obsolete=0 and last_name="Shimazaki" 
return researchers[basics] limit 5

Returned Researchers: 5 (total = 470)


id  last_name first_name  \
0   ur.01214370624.48  Shimazaki     Makiko   
1    ur.0651721117.33  Shimazaki     Megumi   
2   ur.01061154476.35  Shimazaki      Reiri   
3  ur.013166423052.62  Shimazaki     Shingo   
4  ur.011036721776.57  Shimazaki  Toshiyuki   

                                       research_orgs  
0                                                NaN  
1  [{'id': 'grid.268397.1', 'longitude': 131.4687...  
2  [{'id': 'grid.136304.3', 'longitude': 140.1034...  
3  [{'id': 'grid.62167.34', 'longitude': 139.5582...  
4  [{'id': 'grid.411497.e', 'longitude': 130.3662...

## 5. Returning results

After the `search` phrase, a query must contain one or more `return`
phrases, specifying the content and format of the information that
should be returned.



### 5.1 Returning Multiple Sources

Multiple results may not be returned in a single `return` phrase.

In [404]:
%%dsldf 
search publications 
return funders limit 5 
return research_orgs limit 5 
return year

Returned Research_orgs: 5
Returned Year: 20
Returned Funders: 5
[Warning] Dataframe created from first available key, but more than one JSON key found: ['research_orgs', 'year', 'funders']


id   count    city_name   latitude  \
0  grid.26999.3d  323521        Tokyo  35.713333   
1  grid.17063.33  294497      Toronto  43.661667   
2  grid.38142.3c  294239    Cambridge  42.377052   
3  grid.214458.e  258058    Ann Arbor  42.278305   
4  grid.19006.3e  250895  Los Angeles  34.072224   

                                    name acronym        types   country_name  \
0                    University of Tokyo      UT  [Education]          Japan   
1                  University of Toronto     NaN  [Education]         Canada   
2                     Harvard University     NaN  [Education]  United States   
3                 University of Michigan      UM  [Education]  United States   
4  University of California, Los Angeles    UCLA  [Education]  United States   

                          linkout  longitude     state_name  
0  [http://www.u-tokyo.ac.jp/en/]  139.76222            NaN  
1       [http://www.utoronto.ca/]  -79.39500        Ontario  
2       [http://www.harvard.edu/]  -71.11665  Massachusetts  
3        [https://www.umich.edu/]  -83.73822       Michigan  
4          [http://www.ucla.edu/] -118.44410     California


### 5.2 Returning Specific Fields

For control over which information from each given `record` will be
returned, a `source` or `entity` name in the `results` phrase can be
optionally followed by a specification of `fields` and `fieldsets` to be
included in the JSON results for each retrieved record.

The fields specification may be an arbitrary list of `field` names
enclosed in brackets (`[`, `]`), with field names separated by a plus
sign (`+`). Minus sign (`-`) can be used to exclude `field` or a
`fieldset` from the result. Field names thus listed within brackets must
be "known" to the DSL, and therefore only a subset of fields may be used
in this syntax (see note below).

In [378]:
%%dsldf 
search grants 
return grants[grant_number + title + language] limit 5

Returned Grants: 5 (total = 5263527)


title language    grant_number
0  APPROACH to Enriching the Real World Evidence ...       en  2018-HRSI-1548
1  Molecular mechanism of DNA double strand break...       en        1301720F
2                     Life as concept and as science       en          M 2734
3  Jet quenching for heavy-ion collisions at the LHC       en          893021
4  Scintillation Light For New Physics with Liqui...       en          892933

In [379]:
%%dsldf 
search clinical_trials 
return clinical_trials [id+ title + acronym + phase] limit 5

Returned Clinical_trials: 5 (total = 555467)


id                                              title    phase  \
0  NCT00249756     Re-Entry MTC for Offenders With MICA Disorders      NaN   
1  NCT00249782  A Phase II, Randomized, Partial-Blind, Paralle...  Phase 2   
2  NCT00249795  A Parallel Randomized Controlled Evaluation of...  Phase 3   
3  NCT00249808  A Multicentre, Open Label Phase IIIb/IV Study ...  Phase 4   
4  NCT00249847  A Feasibility Study of Positron Emission Tomog...      NaN   

    acronym  
0       NaN  
1       NaN  
2  ACTIVE I  
3       NaN  
4       NaN

**Shortcuts: `fieldsets`**

The fields specification may be the name of a pre-defined `fieldset`
(e.g. `extras`, `basics`). These are shortcuts that can be handy when testing out new queries, for example. 

NOTE In general when writing code used in integrations or long-standing extraction scripts it is **best to return specific fields rather that a predefined set**. This has also the advantage of making queries faster by avoiding the extraction of unnecessary data.
    

In [380]:
%%dsldf 
search grants 
return grants [basics] limit 5 

Returned Grants: 5 (total = 5263527)
WARNINGS [2]
Field 'project_num' is deprecated in favor of grant_number. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'title_language' is deprecated in favor of language_title. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


funders         active_year  \
0  [{'id': 'grid.484521.e', 'longitude': -66.6605...              [2021]   
1  [{'id': 'grid.424470.1', 'longitude': 4.370708...              [2021]   
2  [{'id': 'grid.25111.36', 'longitude': 16.35238...  [2021, 2022, 2023]   
3  [{'id': 'grid.270680.b', 'longitude': 4.36367,...  [2021, 2022, 2023]   
4  [{'id': 'grid.270680.b', 'longitude': 4.36367,...  [2021, 2022, 2023]   

                                      original_title  start_year  \
0  APPROACH to Enriching the Real World Evidence ...        2021   
1  Mécanismes moléculaires de la formation et la ...        2021   
2                     Life as concept and as science        2021   
3  Jet quenching for heavy-ion collisions at the LHC        2021   
4  Scintillation Light For New Physics with Liqui...        2021   

      project_num             id                          funding_org_name  \
0  2018-HRSI-1548  grant.8690978  New Brunswick Health Research Foundation   
1        1301720F  grant.8950252              Fund for Scientific Research   
2          M 2734  grant.8715161                 FWF Austrian Science Fund   
3          893021  grant.8963889                       European Commission   
4          892933  grant.8964235                       European Commission   

   start_date title_language  \
0  2021-11-30             en   
1  2021-10-01             en   
2  2021-10-01             en   
3  2021-09-01             en   
4  2021-09-01             en   

                                               title language    end_date  
0  APPROACH to Enriching the Real World Evidence ...       en         NaN  
1  Molecular mechanism of DNA double strand break...       en         NaN  
2                     Life as concept and as science       en  2023-09-30  
3  Jet quenching for heavy-ion collisions at the LHC       en  2023-08-31  
4  Scintillation Light For New Physics with Liqui...       en  2023-08-31

In [381]:
%%dsldf 
search publications 
return publications [basics+times_cited] limit 5 

Returned Publications: 5 (total = 109295145)
WARNINGS [1]
Field 'author_affiliations' is deprecated in favor of authors. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


pages              id  times_cited  \
0  1071-1080  pub.1123548159            0   
1       1-46  pub.1123550575            0   
2    258-276  pub.1122051582            0   
3        NaN  pub.1123390607            0   
4    207-228  pub.1123015862            0   

                                               title       type  year  \
0               § 33c. Vorbehalt der Gegenseitigkeit    chapter  2020   
1                                         Einleitung    chapter  2020   
2                        CHAPTER XIV. Siting Absence    chapter  2020   
3                         Calling Philosophers Names  monograph  2020   
4  The End of the Socialist (An-)Economy, Money, ...    chapter  2020   

                                 author_affiliations  
0                                                NaN  
1                                                NaN  
2  [[{'first_name': 'Nicole', 'last_name': 'Gerva...  
3  [[{'first_name': 'Christopher', 'last_name': '...  
4  [[{'first_name': 'Jurij', 'last_name': 'Murašo...

The fields specification may be an (`all`), to indicate that all fields
available for the given `source` should be returned.

In [382]:
%%dsldf
search publications 
return publications [all] limit 5 

Returned Publications: 5 (total = 109295145)
WARNINGS [10]
Field 'open_access' is deprecated in favor of open_access_categories. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'HRCS_HC' is deprecated in favor of category_hrcs_hc. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'RCDC' is deprecated in favor of category_rcdc. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'author_affiliations' is deprecated in favor of authors. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'category_ua' is deprecated in favor of category_uoa. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'FOR_first' is deprecated in favor of category_for. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'FOR' is deprecated in favor of category_for. Please refer to https://docs.dim

book_title  \
0       Wertpapiererwerbs- und Übernahmegesetz   
1       Wertpapiererwerbs- und Übernahmegesetz   
2                      Women Mobilizing Memory   
3                                          NaN   
4  Cultures of Economy in South-Eastern Europe   

                              open_access_categories                book_doi  \
0  [{'id': 'closed', 'name': 'Closed', 'descripti...   10.9785/9783504386382   
1  [{'id': 'closed', 'name': 'Closed', 'descripti...   10.9785/9783504386382   
2  [{'id': 'closed', 'name': 'Closed', 'descripti...       10.7312/alti19184   
3  [{'id': 'closed', 'name': 'Closed', 'descripti...                     NaN   
4  [{'id': 'closed', 'name': 'Closed', 'descripti...  10.14361/9783839450260   

               id                                              title  \
0  pub.1123548159               § 33c. Vorbehalt der Gegenseitigkeit   
1  pub.1123550575                                         Einleitung   
2  pub.1122051582                        CHAPTER XIV. Siting Absence   
3  pub.1123390607                         Calling Philosophers Names   
4  pub.1123015862  The End of the Socialist (An-)Economy, Money, ...   

   times_cited                         doi      pages  \
0            0   10.9785/9783504386382-043  1071-1080   
1            0   10.9785/9783504386382-005       1-46   
2            0       10.7312/alti19184-016    258-276   
3            0       10.1515/9780691197425        NaN   
4            0  10.14361/9783839450260-012    207-228   

                   publisher date_inserted  year       type        date  \
0    Verlag Dr. Otto Schmidt    2019-12-23  2020    chapter  2020-12-31   
1    Verlag Dr. Otto Schmidt    2019-12-23  2020    chapter  2020-12-31   
2  Columbia University Press    2019-10-26  2020    chapter  2020-12-31   
3                 De Gruyter    2019-12-15  2020  monograph  2020-12-31   
4          Transcript Verlag    2019-12-01  2020    chapter  2020-12-31   

                                             authors  \
0                                                NaN   
1                                                NaN   
2  [{'first_name': 'Nicole', 'last_name': 'Gervas...   
3  [{'first_name': 'Christopher', 'last_name': 'M...   
4  [{'first_name': 'Jurij', 'last_name': 'Murašov...   

                                 author_affiliations      terms   concepts  
0                                                NaN        NaN        NaN  
1                                                NaN        NaN        NaN  
2  [[{'first_name': 'Nicole', 'last_name': 'Gerva...  [absence]  [absence]  
3  [[{'first_name': 'Christopher', 'last_name': '...        NaN        NaN  
4  [[{'first_name': 'Jurij', 'last_name': 'Murašo...      [end]      [end]

### 5.3 Returning Facets

In addition to returning source records matching a query, it is possible
to $facet$ on the [entity](data-entities.ipynb) fields related to a
particular source and return only those entity values as an aggregrated
view of the related source data. This operation is similar to a
$group by$ or $pivot table$.

**Warning** Faceting can return up to a maximum of 1000 results. This is to ensure
adequate performance with all queries. Furthemore, although the `limit`
operator is allowed, the `skip` operator cannot be used.

In [383]:
%%dsldf 
search publications 
    for "coronavirus" 
return research_orgs limit 5

Returned Research_orgs: 5


id  count     state_name                    linkout  \
0  grid.194645.b    892      Hong Kong       [http://www.hku.hk/]   
1  grid.21107.35    676       Maryland     [https://www.jhu.edu/]   
2  grid.25879.31    612   Pennsylvania    [http://www.upenn.edu/]   
3  grid.416738.f    563        Georgia      [http://www.cdc.gov/]   
4  grid.38142.3c    547  Massachusetts  [http://www.harvard.edu/]   

    country_name   latitude         types acronym  longitude  \
0          China  22.283287   [Education]     HKU  114.13708   
1  United States  39.328888   [Education]     JHU  -76.62028   
2  United States  39.952457   [Education]     NaN  -75.19322   
3  United States  33.798817  [Government]     CDC  -84.32560   
4  United States  42.377052   [Education]     NaN  -71.11665   

                                         name     city_name  
0                     University of Hong Kong     Hong Kong  
1                    Johns Hopkins University     Baltimore  
2                  University of Pennsylvania  Philadelphia  
3  Centers for Disease Control and Prevention       Atlanta  
4                          Harvard University     Cambridge

In [384]:
%%dsldf 
search publications 
    for "coronavirus" 
return research_org_countries limit 5
return year limit 5
return category_for limit 5

Returned Category_for: 5
Returned Research_org_countries: 20
Returned Year: 20
[Warning] Dataframe created from first available key, but more than one JSON key found: ['category_for', 'research_org_countries', 'year']


id  count                                    name
0  2211  48186          11 Medical and Health Sciences
1  2206  19903                  06 Biological Sciences
2  3114  17594               1108 Medical Microbiology
3  3053  10857                  1103 Clinical Sciences
4  3177   9930  1117 Public Health and Health Services

For control over the organization and headers of the JSON query results,
the `return` keyword in a return phrase may be followed by the keyword
`in` and then a `group` name for this group of results, where the group
name is enclosed in double quotes(`"`).

Also, one can define `aliases` that replace the defaul JSON fields names with other ones provided by the user. 

See the [official documentation](https://docs.dimensions.ai/dsl/language.html#aliases) for more details about this feature. 

In [405]:
%%dsldf 
search publications 
return in "facets" funders 
return in "facets" research_orgs

Returned Facets: 2


research_orgs  \
0  [{'id': 'grid.26999.3d', 'count': 323521, 'lin...   

                                             funders  
0  [{'id': 'grid.419696.5', 'count': 1884970, 'li...

### 5.4 What the query statistics refer to - sources VS facets

When performing a DSL search, a `_stats` object is return which contains some useful info eg the total number of records available for a search. 

In [386]:
%%dsldf 
search publications
  where year in [2013:2018] and research_orgs="grid.258806.1"
return publications limit 5

Returned Publications: 5 (total = 3798)


pages              id  \
0  18124-18131  pub.1110885950   
1  29200-29209  pub.1110369527   
2          NaN  pub.1110925389   
3  28491-28496  pub.1110271601   
4  43682-43690  pub.1110222625   

                                               title volume        type  year  \
0  Development of Organo-Dispersible Graphene Oxi...      3     article  2018   
1  Anisotropic Crystal Growth, Optical Absorption...    122     article  2018   
2  Nuclear Ab Initio Calculations with the Unitar...    NaN  proceeding  2018   
3  Indium Zinc Oxide Electron Transport Layer for...    122     article  2018   
4  Chalcopyrite ZnSnSb2: A Promising Thermoelectr...     10     article  2018   

                                 author_affiliations issue    journal.id  \
0  [[{'first_name': 'Siewteng', 'last_name': 'Sim...    12  jour.1157000   
1  [[{'first_name': 'Taro', 'last_name': 'Toyoda'...    51  jour.1038386   
2  [[{'first_name': 'T.', 'last_name': 'Miyagi', ...   NaN           NaN   
3  [[{'first_name': 'Liang', 'last_name': 'Wang',...    50  jour.1038386   
4  [[{'first_name': 'Ami', 'last_name': 'Nomura',...    50  jour.1041450   

                         journal.title  
0                            ACS Omega  
1  The Journal of Physical Chemistry C  
2                                  NaN  
3  The Journal of Physical Chemistry C  
4   ACS Applied Materials & Interfaces



It is important to note though that the **total number always refers to the main source, never the facets** one is searching for. 

For example, in this query we return `researchers` linked to publications: 

In [406]:
%%dsldf 
search publications
  where year in [2013:2018] and research_orgs="grid.258806.1"
return researchers limit 5

Returned Researchers: 5


id  count  \
0   ur.01055753603.27    138   
1  ur.011212042763.67    101   
2   ur.01144540527.52     98   
3   ur.07644453127.11     95   
4  ur.016357156077.09     90   

                                       research_orgs last_name   first_name  \
0  [grid.136593.b, grid.410825.a, grid.482504.f, ...    Hayase  Shuzi Shuzi   
1      [grid.462727.2, grid.258806.1, grid.27476.30]    Hikita     Masayuki   
2  [grid.11135.37, grid.177174.3, grid.411485.d, ...        Ma      Ting-Li   
3  [grid.462727.2, grid.4444.0, grid.258806.1, gr...    Kozako   M Kozako M   
4  [grid.16821.3c, grid.1024.7, grid.454850.8, gr...        Lu       Huimin   

                orcid_id  
0                    NaN  
1                    NaN  
2  [0000-0002-3310-459X]  
3                    NaN  
4  [0000-0001-9794-3221]

NOTE: facet results can be 1000 at most (due to performance limitations) so if there are more than 1000 it is not possible to know the total number. 

### 5.5 Paginating Results

At the end of a `return` phrase, the user can specify the maximum number
of results to be returned and the number of top records to skip over
before returning the first result record, for e.g. returning large
result sets page-by-page (i.e. "paging" results) as described below.

This is done using the keyword `limit` followed by the maximum number of
results to return, optionally followed by the keyword `skip` and the
number of results to skip (the offset).

In [388]:
%%dsldf 
search publications return publications limit 10

Returned Publications: 10 (total = 109295145)


pages              id  \
0  1071-1080  pub.1123548159   
1       1-46  pub.1123550575   
2    258-276  pub.1122051582   
3        NaN  pub.1123390607   
4    207-228  pub.1123015862   
5        NaN  pub.1123390647   
6  viii-viii  pub.1123553347   
7    137-153  pub.1123555428   
8     15-220  pub.1122620908   
9    463-732  pub.1122620912   

                                               title       type  year  \
0               § 33c. Vorbehalt der Gegenseitigkeit    chapter  2020   
1                                         Einleitung    chapter  2020   
2                        CHAPTER XIV. Siting Absence    chapter  2020   
3                         Calling Philosophers Names  monograph  2020   
4  The End of the Socialist (An-)Economy, Money, ...    chapter  2020   
5                                        Utopophobia  monograph  2020   
6                              Bearbeiterverzeichnis    chapter  2020   
7                       § 4. Aufgaben und Befugnisse    chapter  2020   
8  § 2 Rechtsformvergleich und besondere Erschein...    chapter  2020   
9  § 6 Laufende Besteuerung der Gesellschaft und ...    chapter  2020   

                                 author_affiliations  
0                                                NaN  
1                                                NaN  
2  [[{'first_name': 'Nicole', 'last_name': 'Gerva...  
3  [[{'first_name': 'Christopher', 'last_name': '...  
4  [[{'first_name': 'Jurij', 'last_name': 'Murašo...  
5  [[{'first_name': 'David', 'last_name': 'Estlun...  
6                                                NaN  
7                                                NaN  
8  [[{'first_name': 'Thomas', 'last_name': 'Muell...  
9  [[{'first_name': 'Petra', 'last_name': 'Eckl',...

If paging information is not provided, the default values
`limit 20 skip 0` are used, so the two following queries are equivalent:

Combining `limit` and `skip` across multiple queries enables paging or
batching of results; e.g. to retrieve 30 grant records divided into 3
pages of 10 records each, the following three queries could be used:

```
return grants limit 10           => get 1st 10 records for page 1 (skip 0, by default)
return grants limit 10 skip 10   => get next 10 for page 2; skip the 10 we already have
return grants limit 10 skip 20   => get another 10 for page 3, for a total of 30
```

### 5.6 Sorting Results

A sort order for the results in a given `return` phrase can be specified
with the keyword `sort by` followed by the name of 
* a `field` (in the
case that a `source` is being requested) 
* an `indicator (aggregation)` (in the case
that one or more facets are being requested). 

 By default, the result set of full text
queries ($search ... for "full text query"$) is sorted by "relevance".
Additionally, it is possible to specify the sort order, using `asc` or
`desc` keywords. By default, descending order is selected.

In [407]:
%%dsldf 
search grants 
    for "nanomaterials"
return grants sort by title desc limit 5 

Returned Grants: 5 (total = 17425)


start_year    end_date                    original_title  start_date  \
0        2019  2022-03-31                                 x  2019-04-01   
1        2012         NaN  Transmissionselektronenmikroskop  2012-01-01   
2        2015         NaN  Transmissionselektronenmikroskop  2015-01-01   
3        2011  2015-06-13                      Snowcontrol.  2011-06-16   
4        2014         NaN                     Röntgenquelle  2014-01-01   

              id title_language                     active_year  \
0  grant.8518592             pl        [2019, 2020, 2021, 2022]   
1  grant.4823271             en                          [2012]   
2  grant.4841519             en                          [2015]   
3  grant.6774902             en  [2011, 2012, 2013, 2014, 2015]   
4  grant.4834305             en                          [2014]   

                                             funders  \
0  [{'id': 'grid.436846.b', 'city_name': 'Krakow'...   
1  [{'id': 'grid.424150.6', 'city_name': 'Bonn', ...   
2  [{'id': 'grid.424150.6', 'city_name': 'Bonn', ...   
3  [{'id': 'grid.425119.a', 'city_name': 'Brussel...   
4  [{'id': 'grid.424150.6', 'city_name': 'Bonn', ...   

                        funding_org_name          project_num  \
0                National Science Center  2018/29/N/ST5/01240   
1             German Research Foundation            220923099   
2             German Research Foundation            280331443   
3  Belgian Federal Science Policy Office             3E120109   
4             German Research Foundation            245513494   

                              title language  
0                                 x       pl  
1  Transmissionselektronenmikroskop       de  
2  Transmissionselektronenmikroskop       en  
3                      Snowcontrol.       en  
4                     Röntgenquelle       de

In [408]:
%%dsldf  
search grants  
    for "nanomaterials"
return grants  sort by relevance desc limit 5

Returned Grants: 5 (total = 17425)


funders   active_year  \
0  [{'id': 'grid.437854.9', 'longitude': -6.24968...  [2012, 2013]   
1  [{'id': 'grid.452912.9', 'longitude': -75.6925...  [2016, 2017]   
2  [{'id': 'grid.452912.9', 'longitude': -75.6925...  [2016, 2017]   
3  [{'id': 'grid.452912.9', 'longitude': -75.6925...  [2014, 2015]   
4  [{'id': 'grid.452912.9', 'longitude': -75.6925...  [2014, 2015]   

                          original_title  start_year    end_date  \
0  Optically-active chiral nanomaterials        2012  2013-05-31   
1                  Polymer Nanomaterials        2016  2017-03-31   
2                  Polymer Nanomaterials        2016  2017-03-31   
3                  Polymer Nanomaterials        2014  2015-03-31   
4                  Polymer Nanomaterials        2014  2015-03-31   

    project_num             id  \
0  11/W.1/I2065  grant.3984032   
1        617153  grant.6973270   
2        617505  grant.6973622   
3        557300  grant.4167216   
4        557542  grant.4168751   

                                    funding_org_name  start_date  \
0                         Science Foundation Ireland  2012-06-01   
1  Natural Sciences and Engineering Research Council  2016-04-01   
2  Natural Sciences and Engineering Research Council  2016-04-01   
3  Natural Sciences and Engineering Research Council  2014-04-01   
4  Natural Sciences and Engineering Research Council  2014-04-01   

  title_language                                  title language  
0             en  Optically-active chiral nanomaterials       en  
1             en                  Polymer Nanomaterials       en  
2             en                  Polymer Nanomaterials       en  
3             en                  Polymer Nanomaterials       en  
4             en                  Polymer Nanomaterials       en

Number of citations per publication

In [409]:
%%dsldf  
search publications
return publications  [doi + times_cited] 
    sort by times_cited limit 5

Returned Publications: 5 (total = 109295145)


times_cited                           doi
0       230371                           NaN
1       196273              10.1038/227680a0
2       177891  10.1016/0003-2697(76)90527-3
3        86003        10.1006/meth.2001.1262
4        81735   10.1103/physrevlett.77.3865

Recent citations per publication.
Note: Recent citation refers to the number of citations accrued in the last two year period. A single value is stored per document and the year window rolls over in July.

In [411]:
%%dsldf 
search publications
return publications [doi + recent_citations]
    sort by recent_citations limit 5

Returned Publications: 5 (total = 109295145)


recent_citations                               doi
0             27944            10.1006/meth.2001.1262
1             20853       10.1103/physrevlett.77.3865
2             20215  10.1176/appi.books.9780890425596
3             18436         10.1191/1478088706qp063oa
4             15617      10.1016/0003-2697(76)90527-3

When a facet is being returned, the `indicator` used in the
`sort` phrase must either be `count` (the default, such that
`sort by count` is unnecessary), or one of the indicators specified in
the `aggregate` phrase, i.e. one whose values are being computed in the
faceting operation. 


In [412]:
%%dsldf 
search publications 
    for "nanomaterials"
return research_orgs 
    aggregate altmetric_median, rcr_avg sort by rcr_avg limit 5 

Returned Research_orgs: 5


id  count     rcr_avg  altmetric_median    city_name   latitude  \
0  grid.11444.34      1  207.350006             338.0     Shanghai  31.211678   
1  grid.11485.39      1  207.350006             338.0       London  51.531322   
2  grid.11642.30      1  207.350006             338.0  Saint-Denis -20.901735   
3  grid.120073.7      1  207.350006             338.0    Cambridge  52.176000   
4  grid.20931.39      1  207.350006             338.0       London  51.536800   

                                 name         types    country_name  \
0  Shanghai Institute of Hypertension    [Facility]           China   
1                  Cancer Research UK   [Nonprofit]  United Kingdom   
2            University of La Réunion   [Education]         Reunion   
3              Addenbrooke's Hospital  [Healthcare]  United Kingdom   
4            Royal Veterinary College   [Education]  United Kingdom   

                                             linkout   longitude acronym  \
0                        [http://www.china-sih.com/]  121.467255     NaN   
1                 [http://www.cancerresearchuk.org/]   -0.106269    CRUK   
2  [http://www.univ-reunion.fr/university-of-reun...   55.484550     NaN   
3      [http://www.cuh.org.uk/addenbrookes-hospital]    0.140000     NaN   
4                            [http://www.rvc.ac.uk/]   -0.134000     RVC   

       state_name  
0             NaN  
1             NaN  
2             NaN  
3  Cambridgeshire  
4             NaN

## 6. Aggregations

In a `return` phrase requesting one or more `facet` results, aggregation
operations to perform during faceting can be specified after the facet
name(s) by using the keyword `aggregate` followed by a comma-separated
list of one or more `indicator` names corresponding to the `source`
being searched.

In [413]:
%%dsldf
search publications 
    where year > 2010 
return research_orgs  
    aggregate rcr_avg, altmetric_median limit 5

Returned Research_orgs: 5


id   count   rcr_avg  altmetric_median     state_name  \
0  grid.17063.33  138610  1.691560               3.0        Ontario   
1  grid.38142.3c  134298  2.211888               5.0  Massachusetts   
2  grid.11899.38  129887  1.045246               2.0            NaN   
3  grid.83440.3b  118825  1.905526               4.0            NaN   
4  grid.26999.3d  117806  1.180117               2.0            NaN   

   city_name   latitude                       name        types  \
0    Toronto  43.661667      University of Toronto  [Education]   
1  Cambridge  42.377052         Harvard University  [Education]   
2  São Paulo -23.563051    University of Sao Paulo  [Education]   
3     London  51.524470  University College London  [Education]   
4      Tokyo  35.713333        University of Tokyo  [Education]   

     country_name                         linkout   longitude acronym  
0          Canada       [http://www.utoronto.ca/]  -79.395000     NaN  
1   United States       [http://www.harvard.edu/]  -71.116650     NaN  
2          Brazil        [http://www5.usp.br/en/]  -46.730103     USP  
3  United Kingdom         [http://www.ucl.ac.uk/]   -0.133982     UCL  
4           Japan  [http://www.u-tokyo.ac.jp/en/]  139.762220      UT

**What are the metrics/aggregations available?** See the data sources documentation for information about available [indicators](https://docs.dimensions.ai/dsl/datasource-publications.html#publications-indicators).  

Alternatively, we can use the 'schema' API ([describe](https://docs.dimensions.ai/dsl/data-sources.html#metadata-api)) to return this information programmatically:

In [395]:
schema = dsl.query("describe schema")
sources = [x for x in schema['sources']]
# for each source name, extract metrics info
for s in sources:
    print("SOURCE:", s)
    for m in schema['sources'][s]['metrics']:
        print("--", schema['sources'][s]['metrics'][m]['name'], " => ", schema['sources'][s]['metrics'][m]['description'], )

SOURCE: publications
-- count  =>  Total count
-- altmetric_median  =>  Median Altmetric attention score
-- altmetric_avg  =>  Altmetric attention score mean
-- citations_total  =>  Aggregated number of citations
-- citations_avg  =>  Arithmetic mean of citations
-- citations_median  =>  Median of citations
-- recent_citations_total  =>  For a given article, in a given year, the number of citations accrued in the last two year period. Single value stored per document, year window rolls over in July.
-- rcr_avg  =>  Arithmetic mean of `relative_citation_ratio` field.
-- fcr_gavg  =>  Geometric mean of `field_citation_ratio` field (note: This field cannot be used for sorting results).
SOURCE: grants
-- count  =>  Total count
-- funding  =>  Total funding amount, in USD.
SOURCE: patents
-- count  =>  Total count
SOURCE: clinical_trials
-- count  =>  Total count
SOURCE: policy_documents
-- count  =>  Total count
SOURCE: researchers
-- count  =>  Total count
SOURCE: organizations
-- count  

**NOTE** In addition to any specified aggregations, `count` is always computed
and reported when facet results are requested.

In [415]:
%%dsldf
search grants 
    for "5g network" 
return funders 
    aggregate count, funding sort by funding limit 5 

Returned Funders: 5


id  count      funding  city_name   latitude  \
0  grid.270680.b    175  834354500.0   Brussels  50.851650   
1  grid.421091.f     68   52650403.0    Swindon  51.567093   
2  grid.457785.c    104   49173505.0  Arlington  38.880580   
3  grid.55047.33      5   47182381.0     Warsaw  52.227455   
4  grid.457810.f     71   24096160.0  Arlington  38.880580   

                                                name   acronym         types  \
0                                European Commission        EC  [Government]   
1  Engineering and Physical Sciences Research Cou...     EPSRC  [Government]   
2  Directorate for Computer & Information Science...  NSF CISE  [Government]   
3       National Centre for Research and Development      NCRD  [Government]   
4                        Directorate for Engineering   NSF ENG  [Government]   

     country_name                                      linkout  longitude  \
0         Belgium           [http://ec.europa.eu/index_en.htm]   4.363670   
1  United Kingdom                   [https://www.epsrc.ac.uk/]  -1.784602   
2   United States  [http://www.nsf.gov/dir/index.jsp?org=CISE] -77.111000   
3          Poland                 [http://www.ncbr.gov.pl/en/]  21.007630   
4   United States   [http://www.nsf.gov/dir/index.jsp?org=ENG] -77.111000   

  state_name  
0        NaN  
1    England  
2   Virginia  
3        NaN  
4   Virginia

Aggregated total number of citations

In [417]:
%%dsldf
search publications
    for "ontologies"
return funders 
    aggregate citations_total 
    sort by citations_total  limit 5

Returned Funders: 5


id  count  citations_total state_name  city_name   latitude  \
0  grid.48336.3a  11902         787593.0   Maryland  Rockville  39.004326   
1  grid.280785.0  11460         761933.0   Maryland   Bethesda  38.997833   
2  grid.280128.1   4379         563695.0   Maryland   Bethesda  38.996967   
3  grid.270680.b  17489         530976.0        NaN   Brussels  50.851650   
4  grid.52788.30   4777         409828.0        NaN     London  51.525867   

                                             name acronym         types  \
0                       National Cancer Institute     NCI  [Government]   
1  National Institute of General Medical Sciences   NIGMS    [Facility]   
2        National Human Genome Research Institute   NHGRI    [Facility]   
3                             European Commission      EC  [Government]   
4                                  Wellcome Trust      WT   [Nonprofit]   

     country_name                                        linkout  longitude  
0   United States                       [http://www.cancer.gov/] -77.101190  
1   United States  [http://www.nigms.nih.gov/Pages/default.aspx] -77.099380  
2   United States                      [https://www.genome.gov/] -77.096930  
3         Belgium             [http://ec.europa.eu/index_en.htm]   4.363670  
4  United Kingdom                   [http://www.wellcome.ac.uk/]  -0.135005

Arithmetic mean number of citations

In [418]:
%%dsldf
search publications
return funders 
    aggregate citations_avg 
    sort by citations_avg limit 5

Returned Funders: 5


id  count  citations_avg   longitude  \
0  grid.478308.0    165     279.181818  -77.039730   
1  grid.453780.d    142     186.457746  -77.039520   
2  grid.478789.d    567     163.790123 -115.299850   
3  grid.417710.4    180     161.838889  -77.203760   
4  grid.429197.0    718     145.075209  -73.982895   

                           linkout        types            state_name  \
0  [http://www.stewart-trust.org/]  [Nonprofit]  District of Columbia   
1           [http://www.abc2.org/]  [Nonprofit]  District of Columbia   
2     [http://www.dwreynolds.org/]      [Other]                Nevada   
3            [http://www.hgsi.com]    [Company]              Maryland   
4           [http://www.hhwf.org/]      [Other]              New York   

                                    name   country_name        city_name  \
0     Alexander & Margaret Stewart Trust  United States  Washington D.C.   
1           Accelerate Brain Cancer Cure  United States  Washington D.C.   
2          Donald W. Reynolds Foundation  United States        Las Vegas   
3  Human Genome Sciences (United States)  United States        Rockville   
4           Helen Hay Whitney Foundation  United States         New City   

   latitude  
0  38.90116  
1  38.90672  
2  36.19046  
3  39.09665  
4  41.15845

Geometric mean of FCR


In [419]:
%%dsldf
search publications
return funders 
    aggregate fcr_gavg limit 5

Returned Funders: 5


id  fcr_gavg    count  \
0  grid.419696.5  2.310093  1884970   
1  grid.270680.b  3.277453   659906   
2  grid.424020.0  2.523333   592418   
3  grid.48336.3a  4.873172   581356   
4  grid.54432.34  2.255033   566646   

                                     linkout   country_name   latitude  \
0  [http://www.nsfc.gov.cn/publish/portal1/]          China  40.005177   
1         [http://ec.europa.eu/index_en.htm]        Belgium  50.851650   
2              [http://www.most.gov.cn/eng/]          China  39.827835   
3                   [http://www.cancer.gov/]  United States  39.004326   
4                   [http://www.jsps.go.jp/]          Japan  35.687160   

          types acronym   longitude  \
0  [Government]    NSFC  116.339830   
1  [Government]      EC    4.363670   
2  [Government]    MOST  116.316284   
3  [Government]     NCI  -77.101190   
4   [Nonprofit]    JSPS  139.740390   

                                                name  city_name state_name  
0       National Natural Science Foundation of China    Beijing        NaN  
1                                European Commission   Brussels        NaN  
2  Ministry of Science and Technology of the Peop...    Beijing        NaN  
3                          National Cancer Institute  Rockville   Maryland  
4         Japan Society for the Promotion of Science      Tokyo        NaN

Median Altmetric Attention Score

In [420]:
%%dsldf 
search publications
return funders aggregate altmetric_median 
    sort by altmetric_median limit 5 

Returned Funders: 5


id  count  altmetric_median  \
0  grid.258806.1      6             306.0   
1  grid.470711.4      2             111.5   
2  grid.443873.f      5             101.0   
3  grid.473856.b      2              51.0   
4  grid.473769.8      1              33.0   

                                linkout    country_name   latitude  \
0  [https://www.kyutech.ac.jp/english/]           Japan  33.894436   
1             [http://www.chss.org.uk/]  United Kingdom  55.946075   
2           [http://www.lungevity.org/]   United States  41.878674   
3            [https://www.acf.hhs.gov/]   United States  38.885940   
4                [http://www.bcan.org/]   United States  38.988724   

          types acronym   longitude                                      name  \
0   [Education]     KIT  130.839200            Kyushu Institute of Technology   
1   [Nonprofit]    CHSS   -3.219597           Chest Heart and Stroke Scotland   
2   [Nonprofit]    LUNG  -87.626480                      LUNGevity Foundation   
3  [Government]     ACF  -77.016370  Administration for Children and Families   
4   [Nonprofit]    BCAN  -77.097880           Bladder Cancer Advocacy Network   

         city_name            state_name  
0       Kitakyushu                   NaN  
1        Edinburgh                   NaN  
2          Chicago              Illinois  
3  Washington D.C.  District of Columbia  
4         Bethesda              Maryland